In [45]:
import numpy as np
import pandas as pd
import h5py
import os
import glob
from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
import matplotlib.dates as md
import warnings
from scipy.interpolate import interp2d
from scipy.interpolate import interp1d
from scipy.interpolate import make_interp_spline
warnings.filterwarnings('ignore')
import scipy.ndimage as ndimage
from matplotlib import cm
from matplotlib.colors import LogNorm
from sklearn import preprocessing
import seaborn as sns
from scipy.stats import pearsonr
from scipy import stats

In [46]:
def simple_read(path):
    with open(path) as f:
        first_line = f.readline()
        header_line = int(first_line[0:-2].split(",")[0])-1
        #all_lines = f.readlines()
    data = pd.read_csv(path, sep=',', skiprows=header_line)
    data.columns = [i.strip() for i in data.columns]
    #date = all_lines[5][0:10].split(',')
    #dt = datetime(int(date[0]), int(date[1]), int(date[2]))

    acc = 0
    boo = False
    for letter in path:
        if letter == '2':
            boo = True
        elif boo and letter == '0':
            acc -= 1
            break
        acc += 1
    #print(path[:acc+4], path[acc+4:acc+6], path[acc+6:acc+8])
    dt = datetime(int(path[acc:acc+4]), int(path[acc+4:acc+6]), int(path[acc+6:acc+8]))
    
    for column in data.keys():
        if 'Time' in column:
            data[column] = dt + pd.to_timedelta(data[column], unit='seconds')
    return data.replace(-999999, np.NaN)

In [47]:
DAY2 = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220602_R0_L1_20230614T111634.ict')
start_index = np.where(DAY2["Time_Start"]==pd.to_datetime("2022-06-02 13:48:00"))
stop_index = np.where(DAY2["Time_Start"]==pd.to_datetime("2022-06-02 14:10:00"))
    
DAY2_subset = DAY2[start_index[0][0]:stop_index[0][0]]
DAY2_subset = DAY2_subset.replace(-888888, 0)
DAY2_subset


,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
8926,2022-06-02 13:48:00,2022-06-02 13:48:01,32.397719,-65.920473,1896.68,0.241,0.453,0.108,0.114,0.008,-0.049
8927,2022-06-02 13:48:01,2022-06-02 13:48:02,32.397400,-65.918900,1897.28,0.241,0.453,0.108,0.114,0.008,-0.049
8928,2022-06-02 13:48:02,2022-06-02 13:48:03,32.397079,-65.917327,1898.01,0.241,0.453,0.108,0.114,0.008,-0.049
8929,2022-06-02 13:48:03,2022-06-02 13:48:04,32.396758,-65.915754,1898.83,0.241,0.453,0.108,0.114,0.008,-0.049
8930,2022-06-02 13:48:04,2022-06-02 13:48:05,32.396435,-65.914182,1899.75,0.241,0.453,0.108,0.114,0.008,-0.049
...,...,...,...,...,...,...,...,...,...,...,...
10241,2022-06-02 14:09:55,2022-06-02 14:09:56,32.272869,-64.969777,2855.32,0.475,0.315,0.002,0.133,0.035,0.004
10242,2022-06-02 14:09:56,2022-06-02 14:09:57,32.273564,-64.970808,2861.28,0.475,0.315,0.002,0.133,0.035,0.004
10243,2022-06-02 14:09:57,2022-06-02 14:09:58,32.274234,-64.971861,2867.15,0.475,0.315,0.002,0.133,0.035,0.004
10244,2022-06-02 14:09:58,2022-06-02 14:09:59,32.274877,-64.972937,2872.99,0.475,0.315,0.002,0.133,0.035,0.004


In [48]:


normal = (DAY2_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY2_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY2_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY2_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY2_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY2_subset['mz44'] = DAY2_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY2_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY2_subset.loc[:, start_col2:stop_col2] = DAY2_subset.loc[:, start_col2:stop_col2].div(normal, axis=0)
DAY2_subset['AMS'] = normal
DAY2_subset



,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
8926,2022-06-02 13:48:00,2022-06-02 13:48:01,32.397719,-65.920473,1896.68,0.260823,0.490260,0.116883,0.123377,0.008658,-0.049,-0.203320,0.924
8927,2022-06-02 13:48:01,2022-06-02 13:48:02,32.397400,-65.918900,1897.28,0.260823,0.490260,0.116883,0.123377,0.008658,-0.049,-0.203320,0.924
8928,2022-06-02 13:48:02,2022-06-02 13:48:03,32.397079,-65.917327,1898.01,0.260823,0.490260,0.116883,0.123377,0.008658,-0.049,-0.203320,0.924
8929,2022-06-02 13:48:03,2022-06-02 13:48:04,32.396758,-65.915754,1898.83,0.260823,0.490260,0.116883,0.123377,0.008658,-0.049,-0.203320,0.924
8930,2022-06-02 13:48:04,2022-06-02 13:48:05,32.396435,-65.914182,1899.75,0.260823,0.490260,0.116883,0.123377,0.008658,-0.049,-0.203320,0.924
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,2022-06-02 14:09:55,2022-06-02 14:09:56,32.272869,-64.969777,2855.32,0.494792,0.328125,0.002083,0.138542,0.036458,0.004,0.008421,0.960
10242,2022-06-02 14:09:56,2022-06-02 14:09:57,32.273564,-64.970808,2861.28,0.494792,0.328125,0.002083,0.138542,0.036458,0.004,0.008421,0.960
10243,2022-06-02 14:09:57,2022-06-02 14:09:58,32.274234,-64.971861,2867.15,0.494792,0.328125,0.002083,0.138542,0.036458,0.004,0.008421,0.960
10244,2022-06-02 14:09:58,2022-06-02 14:09:59,32.274877,-64.972937,2872.99,0.494792,0.328125,0.002083,0.138542,0.036458,0.004,0.008421,0.960


In [49]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY2_subset.GPS_Altitude_THORNHILL.min(), DAY2_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY2_subset['Bins'] = pd.cut(DAY2_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY2_subset_1000 = DAY2_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY2_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY2_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_1000 = DAY2_subset_1000[start_index:stop_index]
DAY1000_subset_1000


,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(106.28, 134.006]",2022-06-02 13:58:08.316455680,2022-06-02 13:58:09.316455680,32.208589,-65.046984,121.847785,0.155840,0.610456,0.012460,0.213358,0.007885,-0.028734,-0.161706,2.412886
"(134.006, 161.732]",2022-06-02 13:58:10.600000000,2022-06-02 13:58:11.600000000,32.212691,-65.043093,148.533600,0.150824,0.574216,0.012107,0.235920,0.026932,0.014680,0.051524,2.584480
"(161.732, 189.459]",2022-06-02 13:58:12.588235264,2022-06-02 13:58:13.588235264,32.218407,-65.041818,174.502647,0.145899,0.598988,0.010303,0.225904,0.018905,0.023941,0.073367,2.499765
"(189.459, 217.185]",2022-06-02 13:58:03.454545664,2022-06-02 13:58:04.454545408,32.224814,-65.054117,203.336364,0.196569,0.599137,0.000672,0.215906,-0.012283,0.042000,0.090209,2.471727
"(217.185, 244.911]",2022-06-02 13:57:54.700000000,2022-06-02 13:57:55.700000000,32.229233,-65.065482,231.925500,0.195178,0.599907,0.001179,0.216408,-0.012671,0.044400,0.095807,2.469000
"(244.911, 272.637]",2022-06-02 13:58:08.000000000,2022-06-02 13:58:09.000000256,32.231133,-65.053264,259.895833,0.197729,0.598495,0.000249,0.215487,-0.011960,0.040000,0.085543,2.474000
"(272.637, 300.363]",2022-06-02 13:58:30.894736896,2022-06-02 13:58:31.894736896,32.235800,-65.035237,283.662895,0.201086,0.596637,-0.000975,0.214276,-0.011024,0.034211,0.072037,2.480579
"(300.363, 328.09]",2022-06-02 13:57:43.642857216,2022-06-02 13:57:44.642857216,32.243336,-65.088576,313.085357,0.052419,0.709383,0.007329,0.235700,-0.004831,-0.011357,0.199405,2.083857
"(328.09, 355.816]",2022-06-02 13:57:48.700000000,2022-06-02 13:57:49.700000000,32.246542,-65.087461,340.071000,-0.011764,0.769072,0.008723,0.238831,-0.004861,-0.034600,0.144913,1.868600


In [50]:

#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3000

start_index = np.where(DAY2_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY2_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_3000 = DAY2_subset_1000[start_index:stop_index]
DAY3000_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(993.518, 1021.245]",2022-06-02 13:58:28.875000064,2022-06-02 13:58:29.875000064,32.265496,-65.181578,1005.609375,0.103536,0.588208,0.049022,0.252963,0.006271,-0.056000,-0.420293,1.859625
"(1021.245, 1048.971]",2022-06-02 13:58:30.875000064,2022-06-02 13:58:31.875000064,32.267893,-65.180104,1036.225000,0.103536,0.588208,0.049022,0.252963,0.006271,-0.056000,-0.420293,1.859625
"(1048.971, 1076.697]",2022-06-02 14:00:25.300000000,2022-06-02 14:00:26.300000000,32.256697,-65.074768,1061.471000,0.126586,0.599420,0.060982,0.203861,0.009151,-0.064400,-0.341219,1.892000
"(1076.697, 1104.423]",2022-06-02 13:58:37.124999936,2022-06-02 13:58:38.124999936,32.275372,-65.176683,1089.870000,0.103536,0.588208,0.049022,0.252963,0.006271,-0.056000,-0.420293,1.859625
"(1104.423, 1132.149]",2022-06-02 13:59:06.666666752,2022-06-02 13:59:07.666666496,32.276005,-65.151618,1119.245556,0.082544,0.645335,0.027726,0.250858,-0.006463,-0.054000,-0.514070,1.535333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(2712.543, 2740.269]",2022-06-02 14:09:22.000000000,2022-06-02 14:09:23.000000000,32.239794,-64.957288,2728.482727,0.537884,0.279070,0.043511,0.132033,0.007502,0.020000,0.027894,1.333000
"(2740.269, 2767.995]",2022-06-02 14:09:30.500000000,2022-06-02 14:09:31.500000000,32.249103,-64.955685,2752.315000,0.537884,0.279070,0.043511,0.132033,0.007502,0.020000,0.027894,1.333000
"(2767.995, 2795.721]",2022-06-02 14:09:38.000000000,2022-06-02 14:09:39.000000000,32.257308,-64.957425,2783.561111,0.528308,0.289971,0.034305,0.133479,0.013937,0.016444,0.023567,1.250111


In [51]:
DAY2b = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220602_R0_L2_20230614T111634.ict')
start_index = np.where(DAY2b["Time_Start"]==pd.to_datetime("2022-06-02 18:36:00"))
stop_index = np.where(DAY2b["Time_Start"]==pd.to_datetime("2022-06-02 18:53:00"))
DAY2b_subset = DAY2b[start_index[0][0]:stop_index[0][0]]
DAY2b_subset = DAY2b_subset.replace(-888888, 0)
DAY2b_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
9180,2022-06-02 18:36:00,2022-06-02 18:36:01,32.312301,-65.009805,191.225,0.669,1.479,-0.051,0.057,0.028,-0.033
9181,2022-06-02 18:36:01,2022-06-02 18:36:02,32.312336,-65.010962,193.480,0.669,1.479,-0.051,0.057,0.028,-0.033
9182,2022-06-02 18:36:02,2022-06-02 18:36:03,32.312332,-65.012116,195.600,0.669,1.479,-0.051,0.057,0.028,-0.033
9183,2022-06-02 18:36:03,2022-06-02 18:36:04,32.312294,-65.013268,197.010,0.669,1.479,-0.051,0.057,0.028,-0.033
9184,2022-06-02 18:36:04,2022-06-02 18:36:05,32.312221,-65.014415,197.440,0.669,1.479,-0.051,0.057,0.028,-0.033
...,...,...,...,...,...,...,...,...,...,...,...
10195,2022-06-02 18:52:55,2022-06-02 18:52:56,32.205808,-64.977280,3678.610,0.836,0.176,0.024,-0.174,-0.012,0.066
10196,2022-06-02 18:52:56,2022-06-02 18:52:57,32.205215,-64.976028,3684.900,0.836,0.176,0.024,-0.174,-0.012,0.066
10197,2022-06-02 18:52:57,2022-06-02 18:52:58,32.204648,-64.974760,3691.340,0.836,0.176,0.024,-0.174,-0.012,0.066
10198,2022-06-02 18:52:58,2022-06-02 18:52:59,32.204104,-64.973478,3697.790,0.836,0.176,0.024,-0.174,-0.012,0.066


In [52]:
normal2 = (DAY2b_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY2b_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY2b_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY2b_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY2b_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY2b_subset['mz44'] = DAY2b_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY2b_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY2b_subset.loc[:, start_col2:stop_col2] = DAY2b_subset.loc[:, start_col2:stop_col2].div(normal2, axis=0)
DAY2b_subset['AMS'] = normal2
DAY2b_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
9180,2022-06-02 18:36:00,2022-06-02 18:36:01,32.312301,-65.009805,191.225,0.306599,0.677819,-0.023373,0.026123,0.012832,-0.033,-0.049327,2.182
9181,2022-06-02 18:36:01,2022-06-02 18:36:02,32.312336,-65.010962,193.480,0.306599,0.677819,-0.023373,0.026123,0.012832,-0.033,-0.049327,2.182
9182,2022-06-02 18:36:02,2022-06-02 18:36:03,32.312332,-65.012116,195.600,0.306599,0.677819,-0.023373,0.026123,0.012832,-0.033,-0.049327,2.182
9183,2022-06-02 18:36:03,2022-06-02 18:36:04,32.312294,-65.013268,197.010,0.306599,0.677819,-0.023373,0.026123,0.012832,-0.033,-0.049327,2.182
9184,2022-06-02 18:36:04,2022-06-02 18:36:05,32.312221,-65.014415,197.440,0.306599,0.677819,-0.023373,0.026123,0.012832,-0.033,-0.049327,2.182
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10195,2022-06-02 18:52:55,2022-06-02 18:52:56,32.205808,-64.977280,3678.610,0.983529,0.207059,0.028235,-0.204706,-0.014118,0.066,0.078947,0.850
10196,2022-06-02 18:52:56,2022-06-02 18:52:57,32.205215,-64.976028,3684.900,0.983529,0.207059,0.028235,-0.204706,-0.014118,0.066,0.078947,0.850
10197,2022-06-02 18:52:57,2022-06-02 18:52:58,32.204648,-64.974760,3691.340,0.983529,0.207059,0.028235,-0.204706,-0.014118,0.066,0.078947,0.850
10198,2022-06-02 18:52:58,2022-06-02 18:52:59,32.204104,-64.973478,3697.790,0.983529,0.207059,0.028235,-0.204706,-0.014118,0.066,0.078947,0.850


In [53]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY2b_subset.GPS_Altitude_THORNHILL.min(), DAY2b_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY2b_subset['Bins'] = pd.cut(DAY2b_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY2b_subset_1000 = DAY2b_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY2b_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY2b_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_1000b = DAY2b_subset_1000[start_index:stop_index]
DAY1000_subset_1000b

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(132.5, 168.217]",2022-06-02 18:36:33.700000000,2022-06-02 18:36:34.700,32.294115,-65.038343,150.110167,0.247774,0.668646,0.036235,0.024961,0.022385,0.002000,0.005566,2.284500
"(168.217, 203.933]",2022-06-02 18:36:19.440000000,2022-06-02 18:36:20.440,32.302282,-65.024822,185.682600,0.280965,0.673850,-0.007224,0.037155,0.015254,-0.018200,-0.015687,2.205520
"(203.933, 239.65]",2022-06-02 18:37:02.000000000,2022-06-02 18:37:03.000,32.265794,-65.038727,217.346923,0.127354,0.650224,0.035426,0.166816,0.020179,0.068000,0.239437,2.230000
"(239.65, 275.366]",2022-06-02 18:37:13.000000000,2022-06-02 18:37:14.000,32.254753,-65.034836,258.630000,0.127354,0.650224,0.035426,0.166816,0.020179,0.068000,0.239437,2.230000
"(275.366, 311.083]",2022-06-02 18:37:23.999999744,2022-06-02 18:37:25.000,32.245202,-65.026887,290.658077,0.083055,0.668156,0.023730,0.209023,0.016037,0.051846,0.400905,2.024615
"(311.083, 346.8]",2022-06-02 18:37:38.000000000,2022-06-02 18:37:39.000,32.237366,-65.011798,330.868333,0.031373,0.689076,0.010084,0.258263,0.011204,0.033000,0.589286,1.785000
"(346.8, 382.516]",2022-06-02 18:37:53.000000000,2022-06-02 18:37:54.000,32.237006,-64.992797,363.542667,0.079544,0.656800,0.008413,0.254190,0.001052,0.021000,0.358241,1.748200
"(382.516, 418.233]",2022-06-02 18:38:07.000000000,2022-06-02 18:38:08.000,32.244580,-64.977394,399.478462,0.151802,0.608387,0.005907,0.248080,-0.014176,0.003000,0.011673,1.693000
"(418.233, 453.949]",2022-06-02 18:38:19.000000000,2022-06-02 18:38:20.000,32.255481,-64.969336,433.617273,0.151802,0.608387,0.005907,0.248080,-0.014176,0.003000,0.011673,1.693000


In [54]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY2b_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY2b_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3000_subset_3000b = DAY2b_subset_1000[start_index:stop_index]
DAY3000_subset_3000b

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(989.698, 1025.415]",2022-06-02 18:41:23.000,2022-06-02 18:41:24.000,32.269884,-64.947261,1010.109286,0.242646,0.598171,-0.000838,0.156117,0.003906,-0.003000,-0.103645,1.450571
"(1025.415, 1061.132]",2022-06-02 18:41:31.500,2022-06-02 18:41:32.500,32.278879,-64.945764,1045.848000,0.068655,0.590846,0.019417,0.331484,-0.010402,-0.043000,-0.434343,1.442000
"(1061.132, 1096.848]",2022-06-02 18:41:40.500,2022-06-02 18:41:41.500,32.288298,-64.947187,1080.382500,0.068655,0.590846,0.019417,0.331484,-0.010402,-0.043000,-0.434343,1.442000
"(1096.848, 1132.565]",2022-06-02 18:41:52.500,2022-06-02 18:41:53.500,32.298942,-64.954631,1113.911250,0.108788,0.636576,0.028347,0.234966,-0.008677,-0.008875,-0.141550,1.217563
"(1132.565, 1168.281]",2022-06-02 18:42:04.500,2022-06-02 18:42:05.500,32.306134,-64.966376,1149.516250,0.160388,0.695371,0.039828,0.110872,-0.006459,0.035000,0.234899,0.929000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(3275.561, 3311.277]",2022-06-02 18:51:48.500,2022-06-02 18:51:49.500,32.275042,-65.008035,3293.825000,0.323725,0.399113,-0.106430,0.325942,0.057650,-0.033000,-0.226027,0.451000
"(3311.277, 3346.994]",2022-06-02 18:51:54.000,2022-06-02 18:51:55.000,32.268640,-65.008707,3329.810000,0.364525,0.376424,-0.058786,0.252812,0.065025,-0.028200,-0.171361,0.491400
"(3346.994, 3382.711]",2022-06-02 18:51:59.000,2022-06-02 18:52:00.000,32.262832,-65.009195,3366.570000,0.425725,0.342391,0.012681,0.143116,0.076087,-0.021000,-0.089362,0.552000


In [55]:
DAY3 = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220603_R0_20230614T111635.ict')
start_index= np.where(DAY3["Time_Start"]==pd.to_datetime("2022-06-03 13:30:00"))
stop_index= np.where(DAY3["Time_Start"]==pd.to_datetime("2022-06-03 15:00:00"))
    
DAY3_subset = DAY3[start_index[0][0]:stop_index[0][0]]
DAY3_subset = DAY3_subset.replace(-888888, 0)
DAY3_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
2467,2022-06-03 13:30:00,2022-06-03 13:30:01,33.872982,-67.052387,2858.90,0.343,0.244,-0.017,0.025,-0.002,0.050
2468,2022-06-03 13:30:01,2022-06-03 13:30:02,33.872534,-67.053832,2861.55,0.343,0.244,-0.017,0.025,-0.002,0.050
2469,2022-06-03 13:30:02,2022-06-03 13:30:03,33.872086,-67.055272,2863.82,0.343,0.244,-0.017,0.025,-0.002,0.050
2470,2022-06-03 13:30:03,2022-06-03 13:30:04,33.871638,-67.056704,2865.57,0.343,0.244,-0.017,0.025,-0.002,0.050
2471,2022-06-03 13:30:04,2022-06-03 13:30:05,33.871187,-67.058127,2866.59,0.343,0.244,-0.017,0.025,-0.002,0.050
...,...,...,...,...,...,...,...,...,...,...,...
7862,2022-06-03 14:59:55,2022-06-03 14:59:56,32.712767,-65.244314,2413.61,0.003,0.199,0.020,0.215,0.022,-0.014
7863,2022-06-03 14:59:56,2022-06-03 14:59:57,32.712103,-65.243151,2408.15,0.003,0.199,0.020,0.215,0.022,-0.014
7864,2022-06-03 14:59:57,2022-06-03 14:59:58,32.711441,-65.241989,2402.02,0.003,0.199,0.020,0.215,0.022,-0.014
7865,2022-06-03 14:59:58,2022-06-03 14:59:59,32.710779,-65.240827,2395.43,0.003,0.199,0.020,0.215,0.022,-0.014


In [56]:
normal3 = (DAY3_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY3_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY3_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY3_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY3_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY3_subset['mz44'] = DAY3_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY3_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY3_subset.loc[:, start_col2:stop_col2] = DAY3_subset.loc[:, start_col2:stop_col2].div(normal3, axis=0)
DAY3_subset['AMS'] = normal3
DAY3_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
2467,2022-06-03 13:30:00,2022-06-03 13:30:01,33.872982,-67.052387,2858.90,0.578415,0.411467,-0.028668,0.042159,-0.003373,0.050,0.145773,0.593
2468,2022-06-03 13:30:01,2022-06-03 13:30:02,33.872534,-67.053832,2861.55,0.578415,0.411467,-0.028668,0.042159,-0.003373,0.050,0.145773,0.593
2469,2022-06-03 13:30:02,2022-06-03 13:30:03,33.872086,-67.055272,2863.82,0.578415,0.411467,-0.028668,0.042159,-0.003373,0.050,0.145773,0.593
2470,2022-06-03 13:30:03,2022-06-03 13:30:04,33.871638,-67.056704,2865.57,0.578415,0.411467,-0.028668,0.042159,-0.003373,0.050,0.145773,0.593
2471,2022-06-03 13:30:04,2022-06-03 13:30:05,33.871187,-67.058127,2866.59,0.578415,0.411467,-0.028668,0.042159,-0.003373,0.050,0.145773,0.593
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7862,2022-06-03 14:59:55,2022-06-03 14:59:56,32.712767,-65.244314,2413.61,0.006536,0.433551,0.043573,0.468410,0.047930,-0.014,-4.666667,0.459
7863,2022-06-03 14:59:56,2022-06-03 14:59:57,32.712103,-65.243151,2408.15,0.006536,0.433551,0.043573,0.468410,0.047930,-0.014,-4.666667,0.459
7864,2022-06-03 14:59:57,2022-06-03 14:59:58,32.711441,-65.241989,2402.02,0.006536,0.433551,0.043573,0.468410,0.047930,-0.014,-4.666667,0.459
7865,2022-06-03 14:59:58,2022-06-03 14:59:59,32.710779,-65.240827,2395.43,0.006536,0.433551,0.043573,0.468410,0.047930,-0.014,-4.666667,0.459


In [57]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY3_subset.GPS_Altitude_THORNHILL.min(), DAY3_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY3_subset['Bins'] = pd.cut(DAY3_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY3_subset_1000 = DAY3_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY3_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY3_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_3 = DAY3_subset_1000[start_index:stop_index]
DAY1000_subset_3


,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(94.825, 149.89]",2022-06-03 13:45:28.789007104,2022-06-03 13:45:29.789007104,34.126959,-67.213208,123.892358,0.226035,0.615974,0.005921,0.151690,0.000381,0.035413,0.061675,2.311074
"(149.89, 204.956]",2022-06-03 13:48:10.320000000,2022-06-03 13:48:11.319999744,33.927509,-67.395937,181.520500,0.269130,0.550642,0.033513,0.140180,0.006535,0.077720,0.106176,2.396220
"(204.956, 260.021]",2022-06-03 13:59:27.145491712,2022-06-03 13:59:28.145491712,34.102741,-67.396328,246.379939,0.178687,0.640530,0.014309,0.163221,0.003253,0.015936,-0.014753,2.349184
"(260.021, 315.086]",2022-06-03 13:53:11.011236096,2022-06-03 13:53:12.011235840,33.997827,-67.380569,280.595730,0.186761,0.662707,0.015648,0.134865,0.000019,0.020461,0.024999,2.143584
"(315.086, 370.151]",2022-06-03 13:54:31.023809536,2022-06-03 13:54:32.023809536,34.037034,-67.328911,340.301310,0.169653,0.677703,0.011598,0.138903,0.002142,0.034857,0.067674,2.140929
"(370.151, 425.217]",2022-06-03 13:54:07.136363520,2022-06-03 13:54:08.136363776,34.095859,-67.060673,402.749318,0.111311,0.699344,0.007809,0.185070,-0.003535,0.028091,0.402068,2.139727
"(425.217, 480.282]",2022-06-03 14:03:16.657894656,2022-06-03 14:03:17.657894656,34.148126,-67.167576,453.531184,-0.007000,0.869809,0.007707,0.134439,-0.004956,-0.065333,0.779749,1.760000
"(480.282, 535.347]",2022-06-03 14:09:48.304114432,2022-06-03 14:09:49.304114688,34.166931,-67.301055,500.509848,0.154862,0.663589,0.010164,0.163994,0.007390,0.029541,0.056488,2.524010
"(535.347, 590.412]",2022-06-03 13:56:08.066666752,2022-06-03 13:56:09.066666752,33.903960,-67.359933,562.995667,0.035211,0.730134,0.023280,0.195741,0.015634,0.025889,-1.516820,1.333667


In [58]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY3_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY3_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY3_subset_3000 = DAY3_subset_1000[start_index:stop_index]
DAY3_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(975.869, 1030.934]",2022-06-03 14:08:35.406249984,2022-06-03 14:08:36.406249984,34.003272,-67.474357,1004.015000,0.219316,0.668005,-0.000037,0.126571,-0.013855,0.007438,0.018737,1.255437
"(1030.934, 1086.0]",2022-06-03 14:10:41.310344960,2022-06-03 14:10:42.310344960,34.016593,-67.498275,1060.067931,0.220623,0.652696,-0.010780,0.155295,-0.017835,-0.004862,-0.085929,1.251207
"(1086.0, 1141.065]",2022-06-03 14:08:44.529411840,2022-06-03 14:08:45.529411840,33.996922,-67.516110,1107.793824,0.217924,0.606992,-0.009246,0.187071,-0.002740,-0.012353,-0.103505,1.261882
"(1141.065, 1196.13]",2022-06-03 13:54:16.100000000,2022-06-03 13:54:17.100000000,34.028908,-67.067302,1171.628000,0.272623,0.543642,-0.010665,0.201421,-0.007020,-0.065800,-0.249039,0.998400
"(1196.13, 1251.195]",2022-06-03 14:10:12.407407360,2022-06-03 14:10:13.407407616,34.158616,-67.043866,1215.513704,0.349202,0.563121,-0.035833,0.138592,-0.015082,-0.083111,-0.267191,0.912148
"(1251.195, 1306.26]",2022-06-03 13:53:27.769230848,2022-06-03 13:53:28.769230592,34.025997,-67.042836,1278.284615,0.310914,0.526074,-0.029312,0.200964,-0.008640,-0.049923,-0.166109,1.034308
"(1306.26, 1361.326]",2022-06-03 13:59:32.500000000,2022-06-03 13:59:33.500000000,34.077405,-67.028103,1336.740625,0.224105,0.447791,-0.052044,0.384272,-0.004124,-0.026500,-0.126642,1.205500
"(1361.326, 1416.391]",2022-06-03 14:08:28.166666496,2022-06-03 14:08:29.166666752,34.157498,-67.012562,1384.985417,0.205005,0.438763,-0.059620,0.420794,-0.004941,-0.034000,-0.164888,1.209333
"(1416.391, 1471.456]",2022-06-03 13:57:09.909090816,2022-06-03 13:57:10.909090816,34.066318,-67.016919,1443.273636,0.229315,0.450253,-0.049977,0.374312,-0.003902,-0.024455,-0.116211,1.204455


In [59]:
DAY7 = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220607_R0_L1_20230614T111635.ict')
start_index = np.where(DAY7["Time_Start"]==pd.to_datetime("2022-06-07 14:32:00"))
stop_index = np.where(DAY7["Time_Start"]==pd.to_datetime("2022-06-07 14:47:00"))
    
DAY7_subset =DAY7[start_index[0][0]:stop_index[0][0]]
DAY7_subset = DAY7_subset.replace(-888888, 0)
DAY7_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
10397,2022-06-07 14:32:00,2022-06-07 14:32:01,32.250498,-64.999220,127.595,1.99,0.94,0.086,0.069,0.008,0.245
10398,2022-06-07 14:32:01,2022-06-07 14:32:02,32.249883,-64.998233,126.585,1.99,0.94,0.086,0.069,0.008,0.245
10399,2022-06-07 14:32:02,2022-06-07 14:32:03,32.249267,-64.997244,125.870,1.99,0.94,0.086,0.069,0.008,0.245
10400,2022-06-07 14:32:03,2022-06-07 14:32:04,32.248648,-64.996254,125.955,1.99,0.94,0.086,0.069,0.008,0.245
10401,2022-06-07 14:32:04,2022-06-07 14:32:05,32.248027,-64.995263,126.990,1.99,0.94,0.086,0.069,0.008,0.245
...,...,...,...,...,...,...,...,...,...,...,...
11292,2022-06-07 14:46:55,2022-06-07 14:46:56,32.255021,-64.999573,2800.660,0.46,0.33,0.027,-0.270,-0.026,0.072
11293,2022-06-07 14:46:56,2022-06-07 14:46:57,32.255098,-64.998248,2805.770,0.46,0.33,0.027,-0.270,-0.026,0.072
11294,2022-06-07 14:46:57,2022-06-07 14:46:58,32.255152,-64.996919,2810.920,0.46,0.33,0.027,-0.270,-0.026,0.072
11295,2022-06-07 14:46:58,2022-06-07 14:46:59,32.255184,-64.995588,2816.140,0.46,0.33,0.027,-0.270,-0.026,0.072


In [60]:
normal7 = (DAY7_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY7_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY7_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY7_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY7_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY7_subset['mz44'] = DAY7_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY7_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY7_subset.loc[:, start_col2:stop_col2] = DAY7_subset.loc[:, start_col2:stop_col2].div(normal7, axis=0)
DAY7_subset['AMS'] = normal7
DAY7_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
10397,2022-06-07 14:32:00,2022-06-07 14:32:01,32.250498,-64.999220,127.595,0.643388,0.303912,0.027805,0.022308,0.002586,0.245,0.123116,3.093
10398,2022-06-07 14:32:01,2022-06-07 14:32:02,32.249883,-64.998233,126.585,0.643388,0.303912,0.027805,0.022308,0.002586,0.245,0.123116,3.093
10399,2022-06-07 14:32:02,2022-06-07 14:32:03,32.249267,-64.997244,125.870,0.643388,0.303912,0.027805,0.022308,0.002586,0.245,0.123116,3.093
10400,2022-06-07 14:32:03,2022-06-07 14:32:04,32.248648,-64.996254,125.955,0.643388,0.303912,0.027805,0.022308,0.002586,0.245,0.123116,3.093
10401,2022-06-07 14:32:04,2022-06-07 14:32:05,32.248027,-64.995263,126.990,0.643388,0.303912,0.027805,0.022308,0.002586,0.245,0.123116,3.093
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11292,2022-06-07 14:46:55,2022-06-07 14:46:56,32.255021,-64.999573,2800.660,0.882917,0.633397,0.051823,-0.518234,-0.049904,0.072,0.156522,0.521
11293,2022-06-07 14:46:56,2022-06-07 14:46:57,32.255098,-64.998248,2805.770,0.882917,0.633397,0.051823,-0.518234,-0.049904,0.072,0.156522,0.521
11294,2022-06-07 14:46:57,2022-06-07 14:46:58,32.255152,-64.996919,2810.920,0.882917,0.633397,0.051823,-0.518234,-0.049904,0.072,0.156522,0.521
11295,2022-06-07 14:46:58,2022-06-07 14:46:59,32.255184,-64.995588,2816.140,0.882917,0.633397,0.051823,-0.518234,-0.049904,0.072,0.156522,0.521


In [61]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY7_subset.GPS_Altitude_THORNHILL.min(), DAY7_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY7_subset['Bins'] = pd.cut(DAY7_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY7_subset_1000 = DAY7_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY7_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY7_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_7 = DAY7_subset_1000[start_index:stop_index]
DAY1000_subset_7


,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(125.87, 152.826]",2022-06-07 14:32:05.300000000,2022-06-07 14:32:06.300,32.247173,-64.994004,134.321500,0.632981,0.295658,0.026218,0.041858,0.003286,0.283100,0.133630,3.324300
"(152.826, 179.781]",2022-06-07 14:32:13.999999744,2022-06-07 14:32:15.000,32.241209,-64.985889,170.347143,0.608696,0.276398,0.022516,0.087474,0.004917,0.372000,0.158163,3.864000
"(179.781, 206.737]",2022-06-07 14:32:27.500000000,2022-06-07 14:32:28.500,32.229147,-64.976789,189.676250,0.608696,0.276398,0.022516,0.087474,0.004917,0.372000,0.158163,3.864000
"(206.737, 233.692]",2022-06-07 14:32:40.000000000,2022-06-07 14:32:41.000,32.216014,-64.972505,219.682000,0.541940,0.231022,0.020853,0.195408,0.010778,0.296000,0.127972,4.268000
"(233.692, 260.648]",2022-06-07 14:32:48.000000000,2022-06-07 14:32:49.000,32.207258,-64.973206,249.774091,0.541940,0.231022,0.020853,0.195408,0.010778,0.296000,0.127972,4.268000
"(260.648, 287.604]",2022-06-07 14:33:01.000000000,2022-06-07 14:33:02.000,32.194130,-64.979359,265.369667,0.542466,0.233262,0.020500,0.193427,0.010344,0.295200,0.128086,4.249000
"(287.604, 314.559]",2022-06-07 14:33:13.500000000,2022-06-07 14:33:14.500,32.184571,-64.990567,303.789500,0.549837,0.264625,0.015566,0.165704,0.004268,0.284000,0.129680,3.983000
"(314.559, 341.515]",2022-06-07 14:33:26.000000000,2022-06-07 14:33:27.000,32.179710,-65.005119,326.685667,0.549837,0.264625,0.015566,0.165704,0.004268,0.284000,0.129680,3.983000
"(341.515, 368.47]",2022-06-07 14:33:38.000000000,2022-06-07 14:33:39.000,32.179533,-65.019885,353.990000,0.558364,0.262063,0.015011,0.157809,0.006752,0.285111,0.125887,4.058556


In [62]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY7_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY7_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY7_subset_3000 = DAY7_subset_1000[start_index:stop_index]
DAY7_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(1015.405, 1042.36]",2022-06-07 14:38:12.000,2022-06-07 14:38:13.000,32.229338,-65.062171,1029.744286,0.432010,0.395250,0.023097,0.150943,-0.001301,0.1430,0.107681,3.074000
"(1042.36, 1069.316]",2022-06-07 14:38:21.000,2022-06-07 14:38:22.000,32.236954,-65.057676,1056.277273,0.432010,0.395250,0.023097,0.150943,-0.001301,0.1430,0.107681,3.074000
"(1069.316, 1096.272]",2022-06-07 14:38:32.000,2022-06-07 14:38:33.000,32.245122,-65.049970,1083.785455,0.424700,0.397914,0.022420,0.155010,-0.000044,0.1420,0.108577,3.084909
"(1096.272, 1123.227]",2022-06-07 14:38:42.500,2022-06-07 14:38:43.500,32.251109,-65.040405,1108.407000,0.351597,0.424546,0.015654,0.195679,0.012523,0.1320,0.117542,3.194000
"(1123.227, 1150.183]",2022-06-07 14:38:52.000,2022-06-07 14:38:53.000,32.254066,-65.030048,1136.586667,0.351597,0.424546,0.015654,0.195679,0.012523,0.1320,0.117542,3.194000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(2659.696, 2686.652]",2022-06-07 14:46:31.000,2022-06-07 14:46:32.000,32.246791,-65.029386,2672.710000,0.889796,0.273469,-0.104762,0.005442,-0.063946,0.0790,0.120795,0.735000
"(2686.652, 2713.608]",2022-06-07 14:46:36.000,2022-06-07 14:46:37.000,32.249473,-65.023645,2699.616000,0.885669,0.489426,-0.010811,-0.308764,-0.055521,0.0748,0.142231,0.606600
"(2713.608, 2740.563]",2022-06-07 14:46:41.000,2022-06-07 14:46:42.000,32.251655,-65.017598,2726.148000,0.882917,0.633397,0.051823,-0.518234,-0.049904,0.0720,0.156522,0.521000


In [63]:
DAY7b = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220607_R0_L2_20230614T111635.ict')
start_index = np.where(DAY7b["Time_Start"]==pd.to_datetime("2022-06-07 19:08:00"))
stop_index = np.where(DAY7b["Time_Start"]==pd.to_datetime("2022-06-07 19:23:00"))
    
DAY7b_subset = DAY7b[start_index[0][0]:stop_index[0][0]]
DAY7b_subset = DAY7b_subset.replace(-888888, 0)
DAY7b_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
10420,2022-06-07 19:08:00,2022-06-07 19:08:01,32.242823,-65.003952,133.320,0.875,0.562,0.050,0.120,0.004,0.077
10421,2022-06-07 19:08:01,2022-06-07 19:08:02,32.243085,-65.002775,133.120,0.875,0.562,0.050,0.120,0.004,0.077
10422,2022-06-07 19:08:02,2022-06-07 19:08:03,32.243347,-65.001596,132.930,0.875,0.562,0.050,0.120,0.004,0.077
10423,2022-06-07 19:08:03,2022-06-07 19:08:04,32.243610,-65.000416,132.720,0.875,0.562,0.050,0.120,0.004,0.077
10424,2022-06-07 19:08:04,2022-06-07 19:08:05,32.243873,-64.999235,132.595,0.875,0.562,0.050,0.120,0.004,0.077
...,...,...,...,...,...,...,...,...,...,...,...
11315,2022-06-07 19:22:55,2022-06-07 19:22:56,32.296480,-65.011792,3518.030,0.538,0.199,0.045,0.271,0.004,0.148
11316,2022-06-07 19:22:56,2022-06-07 19:22:57,32.296519,-65.013093,3524.890,0.538,0.199,0.045,0.271,0.004,0.148
11317,2022-06-07 19:22:57,2022-06-07 19:22:58,32.296518,-65.014396,3531.840,0.538,0.199,0.045,0.271,0.004,0.148
11318,2022-06-07 19:22:58,2022-06-07 19:22:59,32.296479,-65.015699,3538.850,0.538,0.199,0.045,0.271,0.004,0.148


In [64]:
normal7b = (DAY7b_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY7b_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY7b_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY7b_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY7b_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY7b_subset['mz44'] = DAY7b_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY7b_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY7b_subset.loc[:, start_col2:stop_col2] = DAY7b_subset.loc[:, start_col2:stop_col2].div(normal7b, axis=0)
DAY7b_subset['AMS'] = normal7b
DAY7b_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
10420,2022-06-07 19:08:00,2022-06-07 19:08:01,32.242823,-65.003952,133.320,0.543141,0.348852,0.031037,0.074488,0.002483,0.077,0.088000,1.611
10421,2022-06-07 19:08:01,2022-06-07 19:08:02,32.243085,-65.002775,133.120,0.543141,0.348852,0.031037,0.074488,0.002483,0.077,0.088000,1.611
10422,2022-06-07 19:08:02,2022-06-07 19:08:03,32.243347,-65.001596,132.930,0.543141,0.348852,0.031037,0.074488,0.002483,0.077,0.088000,1.611
10423,2022-06-07 19:08:03,2022-06-07 19:08:04,32.243610,-65.000416,132.720,0.543141,0.348852,0.031037,0.074488,0.002483,0.077,0.088000,1.611
10424,2022-06-07 19:08:04,2022-06-07 19:08:05,32.243873,-64.999235,132.595,0.543141,0.348852,0.031037,0.074488,0.002483,0.077,0.088000,1.611
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11315,2022-06-07 19:22:55,2022-06-07 19:22:56,32.296480,-65.011792,3518.030,0.508988,0.188269,0.042573,0.256386,0.003784,0.148,0.275093,1.057
11316,2022-06-07 19:22:56,2022-06-07 19:22:57,32.296519,-65.013093,3524.890,0.508988,0.188269,0.042573,0.256386,0.003784,0.148,0.275093,1.057
11317,2022-06-07 19:22:57,2022-06-07 19:22:58,32.296518,-65.014396,3531.840,0.508988,0.188269,0.042573,0.256386,0.003784,0.148,0.275093,1.057
11318,2022-06-07 19:22:58,2022-06-07 19:22:59,32.296479,-65.015699,3538.850,0.508988,0.188269,0.042573,0.256386,0.003784,0.148,0.275093,1.057


In [65]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY7b_subset.GPS_Altitude_THORNHILL.min(), DAY7b_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY7b_subset['Bins'] = pd.cut(DAY7b_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY7b_subset_1000 = DAY7b_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY7b_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY7b_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_7b = DAY7b_subset_1000[start_index:stop_index]
DAY1000_subset_7b

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(132.595, 166.728]",2022-06-07 19:08:08.764705792,2022-06-07 19:08:09.764706048,32.245613,-64.993812,142.995294,0.545753,0.346484,0.036597,0.069653,0.001514,0.077000,0.086872,1.625824
"(166.728, 200.861]",2022-06-07 19:08:21.500000000,2022-06-07 19:08:22.500000000,32.252466,-64.980849,183.907500,0.565343,0.328728,0.078296,0.033391,-0.005757,0.077000,0.078411,1.737000
"(200.861, 234.994]",2022-06-07 19:08:31.500000000,2022-06-07 19:08:32.500000000,32.261230,-64.974575,218.264583,0.565343,0.328728,0.078296,0.033391,-0.005757,0.077000,0.078411,1.737000
"(234.994, 269.127]",2022-06-07 19:08:42.500000000,2022-06-07 19:08:43.500000000,32.272442,-64.972311,251.997000,0.551672,0.323049,0.064911,0.059087,0.001281,0.074800,0.080495,1.688600
"(269.127, 303.26]",2022-06-07 19:08:51.500000000,2022-06-07 19:08:52.500000000,32.281693,-64.974483,287.190000,0.496990,0.300334,0.011371,0.161873,0.029431,0.066000,0.088829,1.495000
"(303.26, 337.393]",2022-06-07 19:09:01.500000000,2022-06-07 19:09:02.500000000,32.290806,-64.980798,319.254583,0.496990,0.300334,0.011371,0.161873,0.029431,0.066000,0.088829,1.495000
"(337.393, 371.526]",2022-06-07 19:09:12.000000000,2022-06-07 19:09:13.000000000,32.298242,-64.990766,354.736667,0.496990,0.300334,0.011371,0.161873,0.029431,0.066000,0.088829,1.495000
"(371.526, 405.659]",2022-06-07 19:09:23.500000000,2022-06-07 19:09:24.500000000,32.302454,-65.004622,389.557143,0.588634,0.263816,0.006778,0.144943,-0.004171,0.141000,0.124889,1.918000
"(405.659, 439.792]",2022-06-07 19:09:35.000000000,2022-06-07 19:09:36.000000000,32.302506,-65.019551,421.782778,0.588634,0.263816,0.006778,0.144943,-0.004171,0.141000,0.124889,1.918000


In [66]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY7b_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY7b_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY7b_subset_3000 = DAY7b_subset_1000[start_index:stop_index]
DAY7b_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(985.921, 1020.054]",2022-06-07 19:12:28.500,2022-06-07 19:12:29.500,32.296148,-64.998385,1003.411000,0.563830,0.428369,0.017021,-0.006383,-0.002837,0.184000,0.115723,2.820000
"(1020.054, 1054.187]",2022-06-07 19:12:38.500,2022-06-07 19:12:39.500,32.299974,-65.010363,1037.215000,0.563830,0.428369,0.017021,-0.006383,-0.002837,0.184000,0.115723,2.820000
"(1054.187, 1088.32]",2022-06-07 19:12:48.000,2022-06-07 19:12:49.000,32.300554,-65.022498,1071.926667,0.480199,0.390566,0.023170,0.103502,0.002563,0.226778,0.145182,3.278889
"(1088.32, 1122.453]",2022-06-07 19:12:58.500,2022-06-07 19:12:59.500,32.297572,-65.035423,1104.758333,0.456305,0.379765,0.024927,0.134897,0.004106,0.239000,0.153599,3.410000
"(1122.453, 1156.586]",2022-06-07 19:13:10.500,2022-06-07 19:13:11.500,32.290153,-65.047951,1140.011667,0.463343,0.369697,0.023157,0.139087,0.004716,0.235250,0.151827,3.351750
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(3306.969, 3341.102]",2022-06-07 19:22:24.000,2022-06-07 19:22:25.000,32.278726,-64.979771,3325.450000,0.365105,0.158320,-0.059774,0.537964,-0.001616,0.000000,0.000000,0.619000
"(3341.102, 3375.235]",2022-06-07 19:22:29.500,2022-06-07 19:22:30.500,32.283554,-64.983849,3359.553333,0.365105,0.158320,-0.059774,0.537964,-0.001616,0.000000,0.000000,0.619000
"(3375.235, 3409.368]",2022-06-07 19:22:35.000,2022-06-07 19:22:36.000,32.287882,-64.988668,3393.912000,0.365105,0.158320,-0.059774,0.537964,-0.001616,0.000000,0.000000,0.619000


In [67]:
DAY8 = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220608_R0_L2_20230614T111636.ict')
start_index = np.where(DAY8["Time_Start"]==pd.to_datetime("2022-06-08 20:30:00"))
stop_index = np.where(DAY8["Time_Start"]==pd.to_datetime("2022-06-08 20:45:00"))
    
DAY8_subset = DAY8[start_index[0][0]:stop_index[0][0]]
DAY8_subset = DAY8_subset.replace(-888888, 0)
DAY8_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
10668,2022-06-08 20:30:00,2022-06-08 20:30:01,32.267080,-64.998961,152.725,1.050,1.150,0.088,0.330,0.037,0.169
10669,2022-06-08 20:30:01,2022-06-08 20:30:02,32.268158,-64.999178,153.350,1.050,1.150,0.088,0.330,0.037,0.169
10670,2022-06-08 20:30:02,2022-06-08 20:30:03,32.269234,-64.999434,153.870,1.050,1.150,0.088,0.330,0.037,0.169
10671,2022-06-08 20:30:03,2022-06-08 20:30:04,32.270310,-64.999724,154.255,1.050,1.150,0.088,0.330,0.037,0.169
10672,2022-06-08 20:30:04,2022-06-08 20:30:05,32.271382,-65.000051,154.510,1.050,1.150,0.088,0.330,0.037,0.169
...,...,...,...,...,...,...,...,...,...,...,...
11563,2022-06-08 20:44:55,2022-06-08 20:44:56,32.251172,-65.086081,3838.620,0.153,0.308,0.063,0.185,-0.003,0.050
11564,2022-06-08 20:44:56,2022-06-08 20:44:57,32.250122,-65.085465,3847.170,0.153,0.308,0.063,0.185,-0.003,0.050
11565,2022-06-08 20:44:57,2022-06-08 20:44:58,32.249088,-65.084808,3855.660,0.153,0.308,0.063,0.185,-0.003,0.050
11566,2022-06-08 20:44:58,2022-06-08 20:44:59,32.248074,-65.084107,3864.100,0.153,0.308,0.063,0.185,-0.003,0.050


In [68]:
normal8 = (DAY8_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY8_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY8_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY8_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY8_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY8_subset['mz44'] = DAY8_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY8_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY8_subset.loc[:, start_col2:stop_col2] = DAY8_subset.loc[:, start_col2:stop_col2].div(normal8, axis=0)
DAY8_subset['AMS'] = normal8
DAY8_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
10668,2022-06-08 20:30:00,2022-06-08 20:30:01,32.267080,-64.998961,152.725,0.395480,0.433145,0.033145,0.124294,0.013936,0.169,0.160952,2.655
10669,2022-06-08 20:30:01,2022-06-08 20:30:02,32.268158,-64.999178,153.350,0.395480,0.433145,0.033145,0.124294,0.013936,0.169,0.160952,2.655
10670,2022-06-08 20:30:02,2022-06-08 20:30:03,32.269234,-64.999434,153.870,0.395480,0.433145,0.033145,0.124294,0.013936,0.169,0.160952,2.655
10671,2022-06-08 20:30:03,2022-06-08 20:30:04,32.270310,-64.999724,154.255,0.395480,0.433145,0.033145,0.124294,0.013936,0.169,0.160952,2.655
10672,2022-06-08 20:30:04,2022-06-08 20:30:05,32.271382,-65.000051,154.510,0.395480,0.433145,0.033145,0.124294,0.013936,0.169,0.160952,2.655
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11563,2022-06-08 20:44:55,2022-06-08 20:44:56,32.251172,-65.086081,3838.620,0.216714,0.436261,0.089235,0.262040,-0.004249,0.050,0.326797,0.706
11564,2022-06-08 20:44:56,2022-06-08 20:44:57,32.250122,-65.085465,3847.170,0.216714,0.436261,0.089235,0.262040,-0.004249,0.050,0.326797,0.706
11565,2022-06-08 20:44:57,2022-06-08 20:44:58,32.249088,-65.084808,3855.660,0.216714,0.436261,0.089235,0.262040,-0.004249,0.050,0.326797,0.706
11566,2022-06-08 20:44:58,2022-06-08 20:44:59,32.248074,-65.084107,3864.100,0.216714,0.436261,0.089235,0.262040,-0.004249,0.050,0.326797,0.706


In [69]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY8_subset.GPS_Altitude_THORNHILL.min(), DAY8_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY8_subset['Bins'] = pd.cut(DAY8_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY8_subset_1000 = DAY8_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY8_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY8_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_8 = DAY8_subset_1000[start_index:stop_index]
DAY1000_subset_8


,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(147.455, 184.705]",2022-06-08 20:30:28.263158016,2022-06-08 20:30:29.263158016,32.290577,-65.021135,156.621053,0.416402,0.462028,0.022049,0.091829,0.007692,0.151737,0.140931,2.592439
"(184.705, 221.955]",2022-06-08 20:31:07.000000000,2022-06-08 20:31:08.000000000,32.299647,-65.066050,200.296842,0.411602,0.590239,0.000460,0.001381,-0.003683,0.102000,0.114094,2.172000
"(221.955, 259.205]",2022-06-08 20:31:21.500000000,2022-06-08 20:31:22.500000000,32.291629,-65.080346,241.513000,0.415474,0.600046,0.003904,-0.016382,-0.003042,0.096800,0.108127,2.156200
"(259.205, 296.455]",2022-06-08 20:31:32.500000000,2022-06-08 20:31:33.500000000,32.282682,-65.087075,277.773750,0.430960,0.639274,0.017678,-0.087434,-0.000478,0.076000,0.084257,2.093000
"(296.455, 333.705]",2022-06-08 20:31:44.000000000,2022-06-08 20:31:45.000000000,32.272093,-65.089810,314.530000,0.430960,0.639274,0.017678,-0.087434,-0.000478,0.076000,0.084257,2.093000
"(333.705, 370.955]",2022-06-08 20:31:54.500000000,2022-06-08 20:31:55.500000000,32.262361,-65.088187,352.470000,0.405679,0.640013,0.016214,-0.056012,-0.005894,0.095200,0.113578,2.111800
"(370.955, 408.205]",2022-06-08 20:32:04.500000000,2022-06-08 20:32:05.500000000,32.253986,-65.083164,388.985500,0.367757,0.641121,0.014019,-0.008879,-0.014019,0.124000,0.157560,2.140000
"(408.205, 445.455]",2022-06-08 20:32:15.500000000,2022-06-08 20:32:16.500000000,32.246732,-65.074310,425.771250,0.367757,0.641121,0.014019,-0.008879,-0.014019,0.124000,0.157560,2.140000
"(445.455, 482.705]",2022-06-08 20:32:27.000000000,2022-06-08 20:32:28.000000000,32.242060,-65.062413,462.363182,0.271442,0.648039,0.022496,0.062531,-0.004508,0.105545,0.186748,2.151455


In [70]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY8_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY8_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY8_subset_3000 = DAY8_subset_1000[start_index:stop_index]
DAY8_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(1004.206, 1041.456]",2022-06-08 20:35:22.000,2022-06-08 20:35:23.000,32.281197,-65.081842,1020.976364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1041.456, 1078.706]",2022-06-08 20:35:35.000,2022-06-08 20:35:36.000,32.268830,-65.078436,1060.526000,0.391133,0.475858,0.009414,0.104160,0.019435,0.16100,0.125000,3.2930
"(1078.706, 1115.956]",2022-06-08 20:35:48.500,2022-06-08 20:35:49.500,32.257277,-65.070897,1096.747500,0.391133,0.475858,0.009414,0.104160,0.019435,0.16100,0.125000,3.2930
"(1115.956, 1153.206]",2022-06-08 20:36:00.500,2022-06-08 20:36:01.500,32.250081,-65.060047,1134.750833,0.391754,0.500277,0.009310,0.095089,0.003571,0.13025,0.110128,3.0065
"(1153.206, 1190.456]",2022-06-08 20:36:12.500,2022-06-08 20:36:13.500,32.247857,-65.046366,1172.759167,0.391962,0.508416,0.009275,0.092065,-0.001718,0.12000,0.105171,2.9110
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(3276.459, 3313.709]",2022-06-08 20:43:51.000,2022-06-08 20:43:52.000,32.296061,-65.043973,3293.812000,0.285347,0.650386,-0.074550,0.105398,0.033419,0.02800,0.252252,0.3890
"(3313.709, 3350.959]",2022-06-08 20:43:55.500,2022-06-08 20:43:56.500,32.297056,-65.049661,3332.055000,0.285347,0.650386,-0.074550,0.105398,0.033419,0.02800,0.252252,0.3890
"(3350.959, 3388.209]",2022-06-08 20:44:00.000,2022-06-08 20:44:01.000,32.297200,-65.055477,3370.516000,0.429714,0.568958,-0.104312,0.066539,0.039101,0.01360,0.111201,0.4214


In [71]:
DAY10 = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220610_R0_L1_20230614T111636.ict')
start_index = np.where(DAY10["Time_Start"]==pd.to_datetime("2022-06-10 15:17:00"))
stop_index = np.where(DAY10["Time_Start"]==pd.to_datetime("2022-06-10 15:25:00"))
    
DAY10_subset = DAY10[start_index[0][0]:stop_index[0][0]]
DAY10_subset = DAY10_subset.replace(-888888, 0)
DAY10_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
10616,2022-06-10 15:17:00,2022-06-10 15:17:01,32.206691,-64.996969,122.040,0.311,0.857,0.124,-0.052,-0.006,-0.023
10617,2022-06-10 15:17:01,2022-06-10 15:17:02,32.207809,-64.997169,122.105,0.311,0.857,0.124,-0.052,-0.006,-0.023
10618,2022-06-10 15:17:02,2022-06-10 15:17:03,32.208928,-64.997370,122.170,0.311,0.857,0.124,-0.052,-0.006,-0.023
10619,2022-06-10 15:17:03,2022-06-10 15:17:04,32.210048,-64.997569,122.240,0.311,0.857,0.124,-0.052,-0.006,-0.023
10620,2022-06-10 15:17:04,2022-06-10 15:17:05,32.211167,-64.997768,122.315,0.311,0.857,0.124,-0.052,-0.006,-0.023
...,...,...,...,...,...,...,...,...,...,...,...
11091,2022-06-10 15:24:55,2022-06-10 15:24:56,32.207088,-65.031191,3540.260,0.243,0.189,0.017,-0.335,-0.018,0.027
11092,2022-06-10 15:24:56,2022-06-10 15:24:57,32.207061,-65.029663,3546.970,0.243,0.189,0.017,-0.335,-0.018,0.027
11093,2022-06-10 15:24:57,2022-06-10 15:24:58,32.207068,-65.028134,3553.620,0.243,0.189,0.017,-0.335,-0.018,0.027
11094,2022-06-10 15:24:58,2022-06-10 15:24:59,32.207109,-65.026603,3560.200,0.243,0.189,0.017,-0.335,-0.018,0.027


In [72]:
normal10 = (DAY10_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY10_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY10_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY10_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY10_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY10_subset['mz44'] = DAY10_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY10_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY10_subset.loc[:, start_col2:stop_col2] = DAY10_subset.loc[:, start_col2:stop_col2].div(normal10, axis=0)
DAY10_subset['AMS'] = normal10
DAY10_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
10616,2022-06-10 15:17:00,2022-06-10 15:17:01,32.206691,-64.996969,122.040,0.252026,0.694489,0.100486,-0.042139,-0.004862,-0.023,-0.073955,1.234
10617,2022-06-10 15:17:01,2022-06-10 15:17:02,32.207809,-64.997169,122.105,0.252026,0.694489,0.100486,-0.042139,-0.004862,-0.023,-0.073955,1.234
10618,2022-06-10 15:17:02,2022-06-10 15:17:03,32.208928,-64.997370,122.170,0.252026,0.694489,0.100486,-0.042139,-0.004862,-0.023,-0.073955,1.234
10619,2022-06-10 15:17:03,2022-06-10 15:17:04,32.210048,-64.997569,122.240,0.252026,0.694489,0.100486,-0.042139,-0.004862,-0.023,-0.073955,1.234
10620,2022-06-10 15:17:04,2022-06-10 15:17:05,32.211167,-64.997768,122.315,0.252026,0.694489,0.100486,-0.042139,-0.004862,-0.023,-0.073955,1.234
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11091,2022-06-10 15:24:55,2022-06-10 15:24:56,32.207088,-65.031191,3540.260,2.531250,1.968750,0.177083,-3.489583,-0.187500,0.027,0.111111,0.096
11092,2022-06-10 15:24:56,2022-06-10 15:24:57,32.207061,-65.029663,3546.970,2.531250,1.968750,0.177083,-3.489583,-0.187500,0.027,0.111111,0.096
11093,2022-06-10 15:24:57,2022-06-10 15:24:58,32.207068,-65.028134,3553.620,2.531250,1.968750,0.177083,-3.489583,-0.187500,0.027,0.111111,0.096
11094,2022-06-10 15:24:58,2022-06-10 15:24:59,32.207109,-65.026603,3560.200,2.531250,1.968750,0.177083,-3.489583,-0.187500,0.027,0.111111,0.096


In [73]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY10_subset.GPS_Altitude_THORNHILL.min(), DAY10_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY10_subset['Bins'] = pd.cut(DAY10_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY10_subset_1000 = DAY10_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY10_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY10_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_10 = DAY10_subset_1000[start_index:stop_index]
DAY1000_subset_10

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(122.04, 156.487]",2022-06-10 15:17:25.000,2022-06-10 15:17:26.000,32.234724,-65.001980,124.887449,0.250282,0.679913,0.051482,0.008096,0.010227,-0.015306,-0.047696,1.306796
"(156.487, 190.934]",2022-06-10 15:17:52.500,2022-06-10 15:17:53.500,32.265712,-65.007683,173.583333,0.179692,0.654895,0.016198,0.151928,-0.002713,-0.007500,-0.024908,1.693667
"(190.934, 225.381]",2022-06-10 15:17:58.000,2022-06-10 15:17:59.000,32.271642,-65.009690,209.447000,0.165815,0.651902,0.015900,0.173765,-0.007382,-0.007000,-0.023973,1.761000
"(225.381, 259.828]",2022-06-10 15:18:03.000,2022-06-10 15:18:04.000,32.276778,-65.012305,244.213000,0.165815,0.651902,0.015900,0.173765,-0.007382,-0.007000,-0.023973,1.761000
"(259.828, 294.274]",2022-06-10 15:18:07.500,2022-06-10 15:18:08.500,32.281179,-65.015229,277.968750,0.165815,0.651902,0.015900,0.173765,-0.007382,-0.007000,-0.023973,1.761000
"(294.274, 328.721]",2022-06-10 15:18:12.000,2022-06-10 15:18:13.000,32.285265,-65.018732,312.805000,0.165815,0.651902,0.015900,0.173765,-0.007382,-0.007000,-0.023973,1.761000
"(328.721, 363.168]",2022-06-10 15:18:16.500,2022-06-10 15:18:17.500,32.288972,-65.022764,349.491250,0.165815,0.651902,0.015900,0.173765,-0.007382,-0.007000,-0.023973,1.761000
"(363.168, 397.615]",2022-06-10 15:18:20.500,2022-06-10 15:18:21.500,32.291902,-65.026748,383.632500,0.182907,0.637029,0.003459,0.179098,-0.002493,0.003500,0.008972,1.715500
"(397.615, 432.062]",2022-06-10 15:18:24.500,2022-06-10 15:18:25.500,32.294477,-65.031042,417.876250,0.200000,0.622156,-0.008982,0.184431,0.002395,0.014000,0.041916,1.670000


In [74]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY10_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY10_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY10_subset_3000 = DAY10_subset_1000[start_index:stop_index]
DAY10_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(983.212, 1017.659]",2022-06-10 15:19:33.500000000,2022-06-10 15:19:34.500,32.275739,-65.106964,1001.980000,0.289825,0.547368,-0.012632,0.171930,0.003509,0.09300,0.112591,2.8500
"(1017.659, 1052.106]",2022-06-10 15:19:37.500000000,2022-06-10 15:19:38.500,32.271564,-65.108831,1036.932500,0.289825,0.547368,-0.012632,0.171930,0.003509,0.09300,0.112591,2.8500
"(1052.106, 1086.553]",2022-06-10 15:19:41.500000000,2022-06-10 15:19:42.500,32.267226,-65.110161,1072.345000,0.289825,0.547368,-0.012632,0.171930,0.003509,0.09300,0.112591,2.8500
"(1086.553, 1121.0]",2022-06-10 15:19:45.000000256,2022-06-10 15:19:46.000,32.263340,-65.110896,1103.926667,0.289825,0.547368,-0.012632,0.171930,0.003509,0.09300,0.112591,2.8500
"(1121.0, 1155.447]",2022-06-10 15:19:48.500000000,2022-06-10 15:19:49.500,32.259400,-65.111207,1135.605000,0.305586,0.542596,-0.010806,0.159583,0.003041,0.10175,0.121609,2.7475
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(3291.155, 3325.602]",2022-06-10 15:24:24.000000000,2022-06-10 15:24:25.000,32.223151,-65.072506,3308.126000,0.728180,0.503741,0.167082,-0.418953,0.019950,0.06200,0.212329,0.4010
"(3325.602, 3360.049]",2022-06-10 15:24:29.000000000,2022-06-10 15:24:30.000,32.218877,-65.067075,3344.326000,0.728180,0.503741,0.167082,-0.418953,0.019950,0.06200,0.212329,0.4010
"(3360.049, 3394.496]",2022-06-10 15:24:33.500000000,2022-06-10 15:24:34.500,32.215503,-65.061678,3377.567500,0.728180,0.503741,0.167082,-0.418953,0.019950,0.06200,0.212329,0.4010


In [75]:
DAY10b = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220610_R0_L2_20230614T111636.ict')
start_index = np.where(DAY10b["Time_Start"]==pd.to_datetime("2022-06-10 19:30:00"))
stop_index = np.where(DAY10b["Time_Start"]==pd.to_datetime("2022-06-10 20:45:00"))
    
DAY10b_subset = DAY10b[start_index[0][0]:stop_index[0][0]]
DAY10b_subset = DAY10b_subset.replace(-888888, 0)
DAY10b_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
7182,2022-06-10 19:30:00,2022-06-10 19:30:01,31.639092,-65.606420,113.375,0.248,1.358,0.034,0.269,-0.004,0.026
7183,2022-06-10 19:30:01,2022-06-10 19:30:02,31.639446,-65.607515,113.225,0.248,1.358,0.034,0.269,-0.004,0.026
7184,2022-06-10 19:30:02,2022-06-10 19:30:03,31.639806,-65.608607,113.065,0.248,1.358,0.034,0.269,-0.004,0.026
7185,2022-06-10 19:30:03,2022-06-10 19:30:04,31.640170,-65.609698,112.970,0.248,1.358,0.034,0.269,-0.004,0.026
7186,2022-06-10 19:30:04,2022-06-10 19:30:05,31.640539,-65.610788,112.985,0.248,1.358,0.034,0.269,-0.004,0.026
...,...,...,...,...,...,...,...,...,...,...,...
11677,2022-06-10 20:44:55,2022-06-10 20:44:56,32.077748,-64.914861,1636.660,0.569,0.863,0.009,0.570,0.041,0.124
11678,2022-06-10 20:44:56,2022-06-10 20:44:57,32.078715,-64.913869,1630.380,0.569,0.863,0.009,0.570,0.041,0.124
11679,2022-06-10 20:44:57,2022-06-10 20:44:58,32.079681,-64.912880,1624.290,0.569,0.863,0.009,0.570,0.041,0.124
11680,2022-06-10 20:44:58,2022-06-10 20:44:59,32.080646,-64.911895,1618.170,0.569,0.863,0.009,0.570,0.041,0.124


In [76]:
normal10b = (DAY10b_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY10b_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY10b_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY10b_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY10b_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY10b_subset['mz44'] = DAY10b_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY10b_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY10b_subset.loc[:, start_col2:stop_col2] = DAY10b_subset.loc[:, start_col2:stop_col2].div(normal10b, axis=0)
DAY10b_subset['AMS'] = normal10b
DAY10b_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
7182,2022-06-10 19:30:00,2022-06-10 19:30:01,31.639092,-65.606420,113.375,0.130184,0.712861,0.017848,0.141207,-0.002100,0.026,0.104839,1.905
7183,2022-06-10 19:30:01,2022-06-10 19:30:02,31.639446,-65.607515,113.225,0.130184,0.712861,0.017848,0.141207,-0.002100,0.026,0.104839,1.905
7184,2022-06-10 19:30:02,2022-06-10 19:30:03,31.639806,-65.608607,113.065,0.130184,0.712861,0.017848,0.141207,-0.002100,0.026,0.104839,1.905
7185,2022-06-10 19:30:03,2022-06-10 19:30:04,31.640170,-65.609698,112.970,0.130184,0.712861,0.017848,0.141207,-0.002100,0.026,0.104839,1.905
7186,2022-06-10 19:30:04,2022-06-10 19:30:05,31.640539,-65.610788,112.985,0.130184,0.712861,0.017848,0.141207,-0.002100,0.026,0.104839,1.905
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11677,2022-06-10 20:44:55,2022-06-10 20:44:56,32.077748,-64.914861,1636.660,0.277290,0.420565,0.004386,0.277778,0.019981,0.124,0.217926,2.052
11678,2022-06-10 20:44:56,2022-06-10 20:44:57,32.078715,-64.913869,1630.380,0.277290,0.420565,0.004386,0.277778,0.019981,0.124,0.217926,2.052
11679,2022-06-10 20:44:57,2022-06-10 20:44:58,32.079681,-64.912880,1624.290,0.277290,0.420565,0.004386,0.277778,0.019981,0.124,0.217926,2.052
11680,2022-06-10 20:44:58,2022-06-10 20:44:59,32.080646,-64.911895,1618.170,0.277290,0.420565,0.004386,0.277778,0.019981,0.124,0.217926,2.052


In [77]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY10b_subset.GPS_Altitude_THORNHILL.min(), DAY10b_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY10b_subset['Bins'] = pd.cut(DAY10b_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY10b_subset_1000 = DAY10b_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY10b_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY10b_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_10b = DAY10b_subset_1000[start_index:stop_index]
DAY1000_subset_10b

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(105.665, 154.556]",2022-06-10 19:58:30.078994432,2022-06-10 19:58:31.078994688,31.432104,-65.822952,121.109551,0.163693,0.647290,0.016453,0.170901,0.001663,0.024833,0.077585,2.002068
"(154.556, 203.446]",2022-06-10 20:05:57.967741952,2022-06-10 20:05:58.967741952,31.357706,-65.882896,180.078387,0.221257,0.596836,0.021587,0.159374,0.000946,0.028097,0.063698,2.123065
"(203.446, 252.337]",2022-06-10 20:07:05.333333248,2022-06-10 20:07:06.333333504,31.358167,-65.882335,226.973333,0.207181,0.592380,0.023718,0.176002,0.000719,0.038905,0.092714,2.143238
"(252.337, 301.227]",2022-06-10 20:05:36.318181632,2022-06-10 20:05:37.318181888,31.385854,-65.887268,275.281136,0.119982,0.627095,0.007053,0.248784,-0.002914,0.027727,0.066468,2.132091
"(301.227, 350.118]",2022-06-10 20:00:42.809523712,2022-06-10 20:00:43.809523712,31.488493,-65.882240,325.297857,0.128684,0.616304,0.016320,0.241519,-0.002826,0.031952,0.081329,2.198952
"(350.118, 399.008]",2022-06-10 19:59:54.944444416,2022-06-10 19:59:55.944444416,31.522585,-65.877189,375.466944,0.130838,0.633858,0.002396,0.236615,-0.003707,0.009222,0.025111,2.114222
"(399.008, 447.899]",2022-06-10 20:00:48.157894656,2022-06-10 20:00:49.157894912,31.494981,-65.885690,424.339474,0.125893,0.637421,-0.002309,0.240588,-0.001593,0.007000,0.015145,2.063053
"(447.899, 496.789]",2022-06-10 20:00:51.368421120,2022-06-10 20:00:52.368421120,31.501844,-65.883638,473.128158,0.135918,0.626921,0.005422,0.220363,0.011376,0.018579,0.053527,2.030000
"(496.789, 545.68]",2022-06-10 20:02:18.277777920,2022-06-10 20:02:19.277777920,31.488428,-65.880187,521.287778,0.117705,0.616208,0.016363,0.243587,0.006138,0.008444,0.004567,2.030056


In [78]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY10b_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY10b_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY10b_subset_3000 = DAY10b_subset_1000[start_index:stop_index]
DAY10b_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(985.695, 1034.585]",2022-06-10 19:56:57.444444416,2022-06-10 19:56:58.444444672,31.577510,-65.866113,1011.126667,0.211881,0.587910,-0.000890,0.180424,0.020674,0.053611,0.138195,1.907722
"(1034.585, 1083.476]",2022-06-10 20:02:20.000000000,2022-06-10 20:02:21.000000000,31.495778,-65.884377,1057.859474,0.176003,0.590128,0.016683,0.204514,0.012672,0.060000,0.181562,1.942263
"(1083.476, 1132.367]",2022-06-10 20:01:53.000000000,2022-06-10 20:01:54.000000000,31.495281,-65.887695,1106.609524,0.168955,0.590001,0.019424,0.210765,0.010855,0.061714,0.190013,1.959714
"(1132.367, 1181.257]",2022-06-10 19:58:45.526315776,2022-06-10 19:58:46.526315776,31.560645,-65.878497,1155.327895,0.151694,0.597921,0.010150,0.232028,0.008207,0.063105,0.203194,2.062684
"(1181.257, 1230.148]",2022-06-10 20:10:53.471698176,2022-06-10 20:10:54.471698176,31.242551,-65.928577,1220.275849,0.225118,0.556690,0.021110,0.182606,0.014476,0.034632,0.103707,2.113368
"(1230.148, 1279.038]",2022-06-10 20:12:47.765765888,2022-06-10 20:12:48.765765632,31.218992,-65.934035,1242.383784,0.213141,0.539310,0.017498,0.221104,0.008947,0.048229,0.129410,2.144143
"(1279.038, 1327.929]",2022-06-10 20:04:52.310344704,2022-06-10 20:04:53.310344704,31.329156,-65.912461,1300.516207,0.253496,0.497705,0.019889,0.229936,-0.001025,0.050034,0.093951,2.169759
"(1327.929, 1376.819]",2022-06-10 20:02:45.809523712,2022-06-10 20:02:46.809523712,31.394310,-65.898013,1353.992857,0.271251,0.504335,0.004402,0.212636,0.007376,0.062524,0.105577,2.174381
"(1376.819, 1425.71]",2022-06-10 20:03:59.777777664,2022-06-10 20:04:00.777777920,31.349431,-65.893593,1402.514074,0.280281,0.495750,-0.003606,0.212937,0.014637,0.068667,0.112086,2.171556


In [79]:
DAY11 = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220611_R0_L1_20230614T111637.ict')
start_index = np.where(DAY11["Time_Start"]==pd.to_datetime("2022-06-11 15:24:00"))
stop_index = np.where(DAY11["Time_Start"]==pd.to_datetime("2022-06-11 15:42:00"))
    
DAY11_subset = DAY11[start_index[0][0]:stop_index[0][0]]
DAY11_subset = DAY11_subset.replace(-888888, 0)
DAY11_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
10800,2022-06-11 15:24:00,2022-06-11 15:24:01,32.234864,-64.977277,137.985,0.426,1.496,0.042,0.350,-0.004,-0.003
10801,2022-06-11 15:24:01,2022-06-11 15:24:02,32.233969,-64.976649,138.245,0.426,1.496,0.042,0.350,-0.004,-0.003
10802,2022-06-11 15:24:02,2022-06-11 15:24:03,32.233058,-64.976063,138.485,0.426,1.496,0.042,0.350,-0.004,-0.003
10803,2022-06-11 15:24:03,2022-06-11 15:24:04,32.232130,-64.975521,138.740,0.426,1.496,0.042,0.350,-0.004,-0.003
10804,2022-06-11 15:24:04,2022-06-11 15:24:05,32.231186,-64.975026,139.035,0.426,1.496,0.042,0.350,-0.004,-0.003
...,...,...,...,...,...,...,...,...,...,...,...
11875,2022-06-11 15:41:55,2022-06-11 15:41:56,32.259305,-64.971567,4701.700,0.580,0.154,-0.023,0.363,0.017,0.006
11876,2022-06-11 15:41:56,2022-06-11 15:41:57,32.258543,-64.970428,4708.440,0.580,0.154,-0.023,0.363,0.017,0.006
11877,2022-06-11 15:41:57,2022-06-11 15:41:58,32.257745,-64.969326,4714.800,0.580,0.154,-0.023,0.363,0.017,0.006
11878,2022-06-11 15:41:58,2022-06-11 15:41:59,32.256913,-64.968261,4720.960,0.580,0.154,-0.023,0.363,0.017,0.006


In [80]:
normal11 = (DAY11_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY11_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY11_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY11_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY11_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY11_subset['mz44'] = DAY11_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY11_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY11_subset.loc[:, start_col2:stop_col2] = DAY11_subset.loc[:, start_col2:stop_col2].div(normal11, axis=0)
DAY11_subset['AMS'] = normal11
DAY11_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
10800,2022-06-11 15:24:00,2022-06-11 15:24:01,32.234864,-64.977277,137.985,0.184416,0.647619,0.018182,0.151515,-0.001732,-0.003,-0.007042,2.310
10801,2022-06-11 15:24:01,2022-06-11 15:24:02,32.233969,-64.976649,138.245,0.184416,0.647619,0.018182,0.151515,-0.001732,-0.003,-0.007042,2.310
10802,2022-06-11 15:24:02,2022-06-11 15:24:03,32.233058,-64.976063,138.485,0.184416,0.647619,0.018182,0.151515,-0.001732,-0.003,-0.007042,2.310
10803,2022-06-11 15:24:03,2022-06-11 15:24:04,32.232130,-64.975521,138.740,0.184416,0.647619,0.018182,0.151515,-0.001732,-0.003,-0.007042,2.310
10804,2022-06-11 15:24:04,2022-06-11 15:24:05,32.231186,-64.975026,139.035,0.184416,0.647619,0.018182,0.151515,-0.001732,-0.003,-0.007042,2.310
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11875,2022-06-11 15:41:55,2022-06-11 15:41:56,32.259305,-64.971567,4701.700,0.531622,0.141155,-0.021082,0.332722,0.015582,0.006,0.010345,1.091
11876,2022-06-11 15:41:56,2022-06-11 15:41:57,32.258543,-64.970428,4708.440,0.531622,0.141155,-0.021082,0.332722,0.015582,0.006,0.010345,1.091
11877,2022-06-11 15:41:57,2022-06-11 15:41:58,32.257745,-64.969326,4714.800,0.531622,0.141155,-0.021082,0.332722,0.015582,0.006,0.010345,1.091
11878,2022-06-11 15:41:58,2022-06-11 15:41:59,32.256913,-64.968261,4720.960,0.531622,0.141155,-0.021082,0.332722,0.015582,0.006,0.010345,1.091


In [81]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY11_subset.GPS_Altitude_THORNHILL.min(), DAY11_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY11_subset['Bins'] = pd.cut(DAY11_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY11_subset_1000 = DAY11_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY11_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY11_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_11 = DAY11_subset_1000[start_index:stop_index]
DAY1000_subset_11


,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(137.295, 183.191]",2022-06-11 15:24:38.129870080,2022-06-11 15:24:39.129870080,32.204798,-64.988412,141.553312,0.185407,0.694104,0.011404,0.110829,-0.001745,0.017870,0.025294,2.115338
"(183.191, 229.087]",2022-06-11 15:25:26.500000000,2022-06-11 15:25:27.500000000,32.202954,-65.032135,206.207778,0.058041,0.779927,-0.035067,0.198912,-0.001814,-0.019000,-0.197917,1.654000
"(229.087, 274.983]",2022-06-11 15:25:52.000000000,2022-06-11 15:25:53.000000000,32.228031,-65.036850,245.256061,0.109800,0.708723,-0.009765,0.188183,0.003059,-0.041091,-0.219577,1.683455
"(274.983, 320.88]",2022-06-11 15:26:15.000000000,2022-06-11 15:26:16.000000000,32.246544,-65.020245,296.365000,0.139596,0.670876,0.008729,0.186300,-0.005501,-0.024462,-0.108998,1.673077
"(320.88, 366.776]",2022-06-11 15:26:29.000000000,2022-06-11 15:26:30.000000000,32.249193,-65.003673,343.615333,0.145084,0.664269,0.012590,0.186451,-0.008393,-0.018000,-0.074380,1.668000
"(366.776, 412.672]",2022-06-11 15:26:42.500000000,2022-06-11 15:26:43.500000000,32.245849,-64.988606,392.922917,0.081019,0.676754,0.009184,0.239166,-0.006123,0.014667,0.440639,1.472583
"(412.672, 458.568]",2022-06-11 15:26:58.000000000,2022-06-11 15:26:59.000000000,32.235916,-64.976487,435.027105,0.035259,0.685671,0.006752,0.276819,-0.004501,0.038000,0.808511,1.333000
"(458.568, 504.464]",2022-06-11 15:27:15.500000000,2022-06-11 15:27:16.500000000,32.220892,-64.971716,479.601875,0.098454,0.620257,-0.013144,0.287820,0.006613,-0.009250,0.067236,1.205500
"(504.464, 550.36]",2022-06-11 15:27:29.500000000,2022-06-11 15:27:30.500000000,32.208667,-64.976034,527.068750,0.119518,0.598452,-0.019776,0.291488,0.010318,-0.025000,-0.179856,1.163000


In [82]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY11_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY11_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY11_subset_3000 = DAY11_subset_1000[start_index:stop_index]
DAY11_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(1009.322, 1055.218]",2022-06-11 15:29:30.500000000,2022-06-11 15:29:31.500000000,32.257405,-64.988247,1035.389286,0.198347,0.360631,0.039068,0.391435,0.010518,-0.020000,-0.075758,1.331000
"(1055.218, 1101.114]",2022-06-11 15:29:43.500000000,2022-06-11 15:29:44.500000000,32.250209,-64.974320,1074.022500,0.147919,0.462883,0.025905,0.360430,0.002862,-0.031333,-0.219478,1.133667
"(1101.114, 1147.01]",2022-06-11 15:29:57.500000000,2022-06-11 15:29:58.500000000,32.237558,-64.966375,1125.470625,0.122705,0.514010,0.019324,0.344928,-0.000966,-0.037000,-0.291339,1.035000
"(1147.01, 1192.906]",2022-06-11 15:30:12.500000000,2022-06-11 15:30:13.500000000,32.222628,-64.967036,1170.586429,0.207797,0.523727,-0.000311,0.267590,0.001197,-0.005571,-0.089269,1.051571
"(1192.906, 1238.803]",2022-06-11 15:30:26.000000000,2022-06-11 15:30:27.000000000,32.211306,-64.975437,1213.892308,0.271617,0.531015,-0.015038,0.209586,0.002820,0.018000,0.062284,1.064000
"(1238.803, 1284.699]",2022-06-11 15:30:37.500000000,2022-06-11 15:30:38.500000000,32.205044,-64.986735,1261.504000,0.275262,0.545222,-0.013521,0.193205,-0.000167,0.018400,0.062689,1.065800
"(1284.699, 1330.595]",2022-06-11 15:30:48.000000000,2022-06-11 15:30:49.000000000,32.203083,-64.998878,1309.650000,0.289842,0.602050,-0.007456,0.127679,-0.012116,0.020000,0.064309,1.073000
"(1330.595, 1376.491]",2022-06-11 15:30:59.999999744,2022-06-11 15:31:01.000000000,32.205292,-65.012958,1354.406154,0.289842,0.602050,-0.007456,0.127679,-0.012116,0.020000,0.064309,1.073000
"(1376.491, 1422.387]",2022-06-11 15:31:12.500000000,2022-06-11 15:31:13.500000000,32.212001,-65.026047,1397.715000,0.284707,0.646470,-0.031246,0.103532,-0.003463,0.002667,0.009204,1.130333


In [83]:
DAY11b = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220611_R0_L2_20230614T111637.ict')
start_index = np.where(DAY11b["Time_Start"]==pd.to_datetime("2022-06-11 20:00:00"))
stop_index = np.where(DAY11b["Time_Start"]==pd.to_datetime("2022-06-11 20:30:00"))
    
DAY11b_subset = DAY11b[start_index[0][0]:stop_index[0][0]]
DAY11b_subset = DAY11b_subset.replace(-888888, 0)
DAY11b_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
9350,2022-06-11 20:00:00,2022-06-11 20:00:01,30.851085,-64.222657,132.360,0.393,1.633,0.020,0.140,0.016,0.003
9351,2022-06-11 20:00:01,2022-06-11 20:00:02,30.851840,-64.221712,132.715,0.393,1.633,0.020,0.140,0.016,0.003
9352,2022-06-11 20:00:02,2022-06-11 20:00:03,30.852625,-64.220800,133.040,0.393,1.633,0.020,0.140,0.016,0.003
9353,2022-06-11 20:00:03,2022-06-11 20:00:04,30.853437,-64.219921,133.300,0.393,1.633,0.020,0.140,0.016,0.003
9354,2022-06-11 20:00:04,2022-06-11 20:00:05,30.854278,-64.219077,133.540,0.393,1.633,0.020,0.140,0.016,0.003
...,...,...,...,...,...,...,...,...,...,...,...
11145,2022-06-11 20:29:55,2022-06-11 20:29:56,31.260152,-64.521769,3323.730,0.449,0.322,0.002,0.176,0.003,0.022
11146,2022-06-11 20:29:56,2022-06-11 20:29:57,31.261412,-64.522059,3325.020,0.300,0.233,-0.055,0.148,0.018,-0.016
11147,2022-06-11 20:29:57,2022-06-11 20:29:58,31.262673,-64.522348,3326.270,0.300,0.233,-0.055,0.148,0.018,-0.016
11148,2022-06-11 20:29:58,2022-06-11 20:29:59,31.263934,-64.522638,3327.430,0.300,0.233,-0.055,0.148,0.018,-0.016


In [84]:
normal11b = (DAY11b_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY11b_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY11b_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY11b_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY11b_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY11b_subset['mz44'] = DAY11b_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY11b_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY11b_subset.loc[:, start_col2:stop_col2] = DAY11b_subset.loc[:, start_col2:stop_col2].div(normal11b, axis=0)
DAY11b_subset['AMS'] = normal11b
DAY11b_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
9350,2022-06-11 20:00:00,2022-06-11 20:00:01,30.851085,-64.222657,132.360,0.178474,0.741599,0.009083,0.063579,0.007266,0.003,0.007634,2.202
9351,2022-06-11 20:00:01,2022-06-11 20:00:02,30.851840,-64.221712,132.715,0.178474,0.741599,0.009083,0.063579,0.007266,0.003,0.007634,2.202
9352,2022-06-11 20:00:02,2022-06-11 20:00:03,30.852625,-64.220800,133.040,0.178474,0.741599,0.009083,0.063579,0.007266,0.003,0.007634,2.202
9353,2022-06-11 20:00:03,2022-06-11 20:00:04,30.853437,-64.219921,133.300,0.178474,0.741599,0.009083,0.063579,0.007266,0.003,0.007634,2.202
9354,2022-06-11 20:00:04,2022-06-11 20:00:05,30.854278,-64.219077,133.540,0.178474,0.741599,0.009083,0.063579,0.007266,0.003,0.007634,2.202
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11145,2022-06-11 20:29:55,2022-06-11 20:29:56,31.260152,-64.521769,3323.730,0.471639,0.338235,0.002101,0.184874,0.003151,0.022,0.048998,0.952
11146,2022-06-11 20:29:56,2022-06-11 20:29:57,31.261412,-64.522059,3325.020,0.465839,0.361801,-0.085404,0.229814,0.027950,-0.016,-0.053333,0.644
11147,2022-06-11 20:29:57,2022-06-11 20:29:58,31.262673,-64.522348,3326.270,0.465839,0.361801,-0.085404,0.229814,0.027950,-0.016,-0.053333,0.644
11148,2022-06-11 20:29:58,2022-06-11 20:29:59,31.263934,-64.522638,3327.430,0.465839,0.361801,-0.085404,0.229814,0.027950,-0.016,-0.053333,0.644


In [85]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY11b_subset.GPS_Altitude_THORNHILL.min(), DAY11b_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY11b_subset['Bins'] = pd.cut(DAY11b_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY11b_subset_1000 = DAY11b_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY11b_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY11b_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_11b = DAY11b_subset_1000[start_index:stop_index]
DAY1000_subset_11b


,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(108.925, 141.12]",2022-06-11 20:04:07.512096768,2022-06-11 20:04:08.512096768,31.035566,-64.424176,123.488034,0.200989,0.685663,0.015249,0.096796,0.001303,0.029952,0.067661,2.172788
"(141.12, 173.315]",2022-06-11 20:08:18.500000000,2022-06-11 20:08:19.500000000,31.221775,-64.662991,155.437500,0.052200,0.736764,0.041014,0.173005,-0.002983,0.018000,0.257143,1.341000
"(173.315, 205.51]",2022-06-11 20:08:22.500000000,2022-06-11 20:08:23.500000000,31.222161,-64.667873,188.907500,0.052200,0.736764,0.041014,0.173005,-0.002983,0.018000,0.257143,1.341000
"(205.51, 237.706]",2022-06-11 20:08:26.500000000,2022-06-11 20:08:27.500000000,31.222612,-64.672638,223.777500,0.299712,0.688761,0.020893,-0.017291,0.007925,0.028000,0.067308,1.388000
"(237.706, 269.901]",2022-06-11 20:08:30.000000000,2022-06-11 20:08:30.999999744,31.223105,-64.676702,256.585000,0.299712,0.688761,0.020893,-0.017291,0.007925,0.028000,0.067308,1.388000
"(269.901, 302.096]",2022-06-11 20:08:33.500000000,2022-06-11 20:08:34.500000000,31.223615,-64.680669,288.658750,0.299712,0.688761,0.020893,-0.017291,0.007925,0.028000,0.067308,1.388000
"(302.096, 334.291]",2022-06-11 20:08:38.500000000,2022-06-11 20:08:39.500000000,31.224804,-64.686121,320.795833,0.299712,0.688761,0.020893,-0.017291,0.007925,0.028000,0.067308,1.388000
"(334.291, 366.486]",2022-06-11 20:08:43.500000000,2022-06-11 20:08:44.500000000,31.226848,-64.691161,351.048750,0.299712,0.688761,0.020893,-0.017291,0.007925,0.028000,0.067308,1.388000
"(366.486, 398.681]",2022-06-11 20:08:46.999999744,2022-06-11 20:08:48.000000000,31.228959,-64.694189,384.708333,0.299712,0.688761,0.020893,-0.017291,0.007925,0.028000,0.067308,1.388000


In [86]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY11b_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY11b_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY11b_subset_3000 = DAY11b_subset_1000[start_index:stop_index]
DAY11b_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(1010.389, 1042.584]",2022-06-11 20:10:22.999999744,2022-06-11 20:10:24.000000000,31.228788,-64.661029,1025.980000,0.227942,0.523308,0.011302,0.231885,0.005562,0.038333,0.065515,1.809000
"(1042.584, 1074.779]",2022-06-11 20:10:26.500000000,2022-06-11 20:10:27.500000000,31.226313,-64.664300,1058.642500,0.206002,0.590430,0.002433,0.209246,-0.008110,-0.017000,-0.066929,1.233000
"(1074.779, 1106.975]",2022-06-11 20:10:30.000000000,2022-06-11 20:10:30.999999744,31.224284,-64.667959,1091.333333,0.206002,0.590430,0.002433,0.209246,-0.008110,-0.017000,-0.066929,1.233000
"(1106.975, 1139.17]",2022-06-11 20:10:33.500000000,2022-06-11 20:10:34.500000000,31.222815,-64.671970,1121.872500,0.206002,0.590430,0.002433,0.209246,-0.008110,-0.017000,-0.066929,1.233000
"(1139.17, 1171.365]",2022-06-11 20:10:37.000000256,2022-06-11 20:10:38.000000000,31.221921,-64.676211,1153.480000,0.206002,0.590430,0.002433,0.209246,-0.008110,-0.017000,-0.066929,1.233000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(3135.269, 3167.464]",2022-06-11 20:18:06.769230848,2022-06-11 20:18:07.769230848,31.241240,-64.609057,3150.196154,0.333341,0.523184,0.006799,0.162058,-0.025381,0.004692,0.024256,0.647923
"(3167.464, 3199.659]",2022-06-11 20:29:04.500000000,2022-06-11 20:29:05.500000000,31.198084,-64.505758,3184.927500,0.350365,0.737226,-0.063260,-0.002433,-0.021898,0.007000,0.048611,0.411000
"(3199.659, 3231.855]",2022-06-11 20:29:08.500000000,2022-06-11 20:29:09.500000000,31.202882,-64.507282,3218.335000,0.350365,0.737226,-0.063260,-0.002433,-0.021898,0.007000,0.048611,0.411000


In [87]:
DAY13 = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220613_R0_L1_20230614T111637.ict')
start_index = np.where(DAY13["Time_Start"]==pd.to_datetime("2022-06-13 14:30:00"))
stop_index = np.where(DAY13["Time_Start"]==pd.to_datetime("2022-06-13 14:46:00"))
    
DAY13_subset = DAY13[start_index[0][0]:stop_index[0][0]]
DAY13_subset = DAY13_subset.replace(-888888, 0)
DAY13_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
10015,2022-06-13 14:30:00,2022-06-13 14:30:01,32.210354,-65.054053,141.795,0.207,0.760,-0.037,0.166,0.031,-0.012
10016,2022-06-13 14:30:01,2022-06-13 14:30:02,32.209934,-65.052981,141.725,0.207,0.760,-0.037,0.166,0.031,-0.012
10017,2022-06-13 14:30:02,2022-06-13 14:30:03,32.209541,-65.051894,141.615,0.207,0.760,-0.037,0.166,0.031,-0.012
10018,2022-06-13 14:30:03,2022-06-13 14:30:04,32.209173,-65.050793,141.460,0.207,0.760,-0.037,0.166,0.031,-0.012
10019,2022-06-13 14:30:04,2022-06-13 14:30:05,32.208832,-65.049680,141.275,0.207,0.760,-0.037,0.166,0.031,-0.012
...,...,...,...,...,...,...,...,...,...,...,...
10970,2022-06-13 14:45:55,2022-06-13 14:45:56,32.197220,-65.046274,4504.770,0.472,0.218,0.043,0.118,0.027,0.009
10971,2022-06-13 14:45:56,2022-06-13 14:45:57,32.196954,-65.044914,4514.670,0.472,0.218,0.043,0.118,0.027,0.009
10972,2022-06-13 14:45:57,2022-06-13 14:45:58,32.196726,-65.043543,4523.980,0.472,0.218,0.043,0.118,0.027,0.009
10973,2022-06-13 14:45:58,2022-06-13 14:45:59,32.196539,-65.042160,4532.940,0.472,0.218,0.043,0.118,0.027,0.009


In [88]:
normal13 = (DAY13_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY13_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY13_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY13_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY13_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY13_subset['mz44'] = DAY13_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY13_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY13_subset.loc[:, start_col2:stop_col2] = DAY13_subset.loc[:, start_col2:stop_col2].div(normal13, axis=0)
DAY13_subset['AMS'] = normal13
DAY13_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
10015,2022-06-13 14:30:00,2022-06-13 14:30:01,32.210354,-65.054053,141.795,0.183673,0.674357,-0.032831,0.147294,0.027507,-0.012,-0.057971,1.127
10016,2022-06-13 14:30:01,2022-06-13 14:30:02,32.209934,-65.052981,141.725,0.183673,0.674357,-0.032831,0.147294,0.027507,-0.012,-0.057971,1.127
10017,2022-06-13 14:30:02,2022-06-13 14:30:03,32.209541,-65.051894,141.615,0.183673,0.674357,-0.032831,0.147294,0.027507,-0.012,-0.057971,1.127
10018,2022-06-13 14:30:03,2022-06-13 14:30:04,32.209173,-65.050793,141.460,0.183673,0.674357,-0.032831,0.147294,0.027507,-0.012,-0.057971,1.127
10019,2022-06-13 14:30:04,2022-06-13 14:30:05,32.208832,-65.049680,141.275,0.183673,0.674357,-0.032831,0.147294,0.027507,-0.012,-0.057971,1.127
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10970,2022-06-13 14:45:55,2022-06-13 14:45:56,32.197220,-65.046274,4504.770,0.537585,0.248292,0.048975,0.134396,0.030752,0.009,0.019068,0.878
10971,2022-06-13 14:45:56,2022-06-13 14:45:57,32.196954,-65.044914,4514.670,0.537585,0.248292,0.048975,0.134396,0.030752,0.009,0.019068,0.878
10972,2022-06-13 14:45:57,2022-06-13 14:45:58,32.196726,-65.043543,4523.980,0.537585,0.248292,0.048975,0.134396,0.030752,0.009,0.019068,0.878
10973,2022-06-13 14:45:58,2022-06-13 14:45:59,32.196539,-65.042160,4532.940,0.537585,0.248292,0.048975,0.134396,0.030752,0.009,0.019068,0.878


In [89]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY13_subset.GPS_Altitude_THORNHILL.min(), DAY13_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY13_subset['Bins'] = pd.cut(DAY13_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY13_subset_1000 = DAY13_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY13_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY13_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_13 = DAY13_subset_1000[start_index:stop_index]
DAY1000_subset_13


,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(138.195, 182.23]",2022-06-13 14:30:32.200000000,2022-06-13 14:30:33.200,32.215823,-65.019495,142.037308,0.127818,0.748561,-0.043264,0.134105,0.032781,0.004769,-0.171372,1.124077
"(182.23, 226.264]",2022-06-13 14:31:11.500000000,2022-06-13 14:31:12.500,32.245854,-64.994298,201.121250,0.202002,0.652687,0.039842,0.090112,0.015358,0.011750,-0.129018,1.331750
"(226.264, 270.299]",2022-06-13 14:31:24.000000256,2022-06-13 14:31:25.000,32.259352,-64.997848,250.047308,0.052950,0.642209,0.058245,0.245840,0.000756,-0.058000,-0.828571,1.322000
"(270.299, 314.334]",2022-06-13 14:31:36.500000000,2022-06-13 14:31:37.500,32.271439,-65.006302,292.679167,0.052950,0.642209,0.058245,0.245840,0.000756,-0.058000,-0.828571,1.322000
"(314.334, 358.368]",2022-06-13 14:31:48.000000000,2022-06-13 14:31:49.000,32.279961,-65.017932,338.120000,0.196938,0.647478,0.043109,0.114939,-0.002464,-0.038364,-0.247240,1.269636
"(358.368, 402.403]",2022-06-13 14:32:01.000000000,2022-06-13 14:32:02.000,32.285610,-65.034070,380.360667,0.228935,0.648649,0.039746,0.085851,-0.003180,-0.034000,-0.118056,1.258000
"(402.403, 446.438]",2022-06-13 14:32:13.000000000,2022-06-13 14:32:14.000,32.287260,-65.050147,423.533333,0.263068,0.719789,0.012004,0.003760,0.001378,-0.024000,-0.084737,1.061000
"(446.438, 490.472]",2022-06-13 14:32:22.000000000,2022-06-13 14:32:23.000,32.285677,-65.062083,469.225556,0.331334,0.862069,-0.043478,-0.160420,0.010495,-0.004000,-0.018100,0.667000
"(490.472, 534.507]",2022-06-13 14:32:33.000000000,2022-06-13 14:32:34.000,32.280467,-65.075342,512.890385,0.331334,0.862069,-0.043478,-0.160420,0.010495,-0.004000,-0.018100,0.667000


In [90]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY13_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY13_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY13_subset_3000 = DAY13_subset_1000[start_index:stop_index]
DAY13_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(1018.888, 1062.923]",2022-06-13 14:34:55.500,2022-06-13 14:34:56.500000000,32.220498,-64.995521,1042.884167,0.047330,0.658981,-0.020631,0.322816,-0.008495,-0.047000,-1.205128,0.824000
"(1062.923, 1106.957]",2022-06-13 14:35:07.500,2022-06-13 14:35:08.500000000,32.233581,-64.989205,1085.528333,0.047330,0.658981,-0.020631,0.322816,-0.008495,-0.047000,-1.205128,0.824000
"(1106.957, 1150.992]",2022-06-13 14:35:20.000,2022-06-13 14:35:21.000000000,32.248197,-64.987717,1131.330000,0.235609,0.605884,-0.044306,0.215033,-0.012220,-0.010615,-0.197848,0.979692
"(1150.992, 1195.027]",2022-06-13 14:35:35.000,2022-06-13 14:35:36.000000000,32.265068,-64.993100,1172.945882,0.269841,0.596230,-0.048611,0.195437,-0.012897,-0.004000,-0.014706,1.008000
"(1195.027, 1239.061]",2022-06-13 14:35:49.500,2022-06-13 14:35:50.500000000,32.278863,-65.004578,1216.515000,0.110262,0.638907,0.020915,0.230251,-0.000336,-0.003167,-0.037173,0.933833
"(1239.061, 1283.096]",2022-06-13 14:36:04.000,2022-06-13 14:36:05.000000000,32.288351,-65.020848,1261.682353,0.078346,0.647443,0.034820,0.237214,0.002176,-0.003000,-0.041667,0.919000
"(1283.096, 1327.131]",2022-06-13 14:36:18.000,2022-06-13 14:36:19.000000000,32.292196,-65.039287,1304.431818,0.208451,0.569066,0.018918,0.189683,0.013882,0.014182,0.060606,0.911636
"(1327.131, 1371.165]",2022-06-13 14:36:32.000,2022-06-13 14:36:33.000000000,32.290319,-65.058117,1351.644118,0.237363,0.551648,0.015385,0.179121,0.016484,0.018000,0.083333,0.910000
"(1371.165, 1415.2]",2022-06-13 14:36:48.500,2022-06-13 14:36:49.500000000,32.280732,-65.077275,1389.921875,0.215918,0.579298,0.047234,0.136971,0.020579,0.002187,0.002657,0.774563


In [91]:
DAY13b = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220613_R0_L2_20230614T111637.ict')
start_index = np.where(DAY13b["Time_Start"]==pd.to_datetime("2022-06-13 19:46:00"))
stop_index = np.where(DAY13b["Time_Start"]==pd.to_datetime("2022-06-13 20:09:00")) 
DAY13b_subset = DAY13b[start_index[0][0]:stop_index[0][0]]
DAY13b_subset = DAY13b_subset.replace(-888888, 0)
DAY13b_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
10610,2022-06-13 19:46:00,2022-06-13 19:46:01,32.251226,-65.007304,106.13,0.122,0.732,0.079,0.011,-0.010,-0.036
10611,2022-06-13 19:46:01,2022-06-13 19:46:02,32.250414,-65.006555,105.59,0.122,0.732,0.079,0.011,-0.010,-0.036
10612,2022-06-13 19:46:02,2022-06-13 19:46:03,32.249605,-65.005808,106.45,0.122,0.732,0.079,0.011,-0.010,-0.036
10613,2022-06-13 19:46:03,2022-06-13 19:46:04,32.248799,-65.005062,108.91,0.122,0.732,0.079,0.011,-0.010,-0.036
10614,2022-06-13 19:46:04,2022-06-13 19:46:05,32.247996,-65.004323,112.60,0.122,0.732,0.079,0.011,-0.010,-0.036
...,...,...,...,...,...,...,...,...,...,...,...
11985,2022-06-13 20:08:55,2022-06-13 20:08:56,32.290180,-64.996666,4558.09,0.299,0.259,-0.013,-0.186,0.032,0.055
11986,2022-06-13 20:08:56,2022-06-13 20:08:57,32.289203,-64.997413,4565.53,0.299,0.259,-0.013,-0.186,0.032,0.055
11987,2022-06-13 20:08:57,2022-06-13 20:08:58,32.288204,-64.998124,4572.98,0.299,0.259,-0.013,-0.186,0.032,0.055
11988,2022-06-13 20:08:58,2022-06-13 20:08:59,32.287185,-64.998799,4580.36,0.098,0.231,0.133,0.113,-0.036,0.018


In [92]:
normal13b = (DAY13b_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY13b_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY13b_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY13b_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY13b_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY13b_subset['mz44'] = DAY13b_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY13b_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY13b_subset.loc[:, start_col2:stop_col2] = DAY13b_subset.loc[:, start_col2:stop_col2].div(normal13b, axis=0)
DAY13b_subset['AMS'] = normal13b
DAY13b_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
10610,2022-06-13 19:46:00,2022-06-13 19:46:01,32.251226,-65.007304,106.13,0.130621,0.783726,0.084582,0.011777,-0.010707,-0.036,-0.295082,0.934
10611,2022-06-13 19:46:01,2022-06-13 19:46:02,32.250414,-65.006555,105.59,0.130621,0.783726,0.084582,0.011777,-0.010707,-0.036,-0.295082,0.934
10612,2022-06-13 19:46:02,2022-06-13 19:46:03,32.249605,-65.005808,106.45,0.130621,0.783726,0.084582,0.011777,-0.010707,-0.036,-0.295082,0.934
10613,2022-06-13 19:46:03,2022-06-13 19:46:04,32.248799,-65.005062,108.91,0.130621,0.783726,0.084582,0.011777,-0.010707,-0.036,-0.295082,0.934
10614,2022-06-13 19:46:04,2022-06-13 19:46:05,32.247996,-65.004323,112.60,0.130621,0.783726,0.084582,0.011777,-0.010707,-0.036,-0.295082,0.934
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11985,2022-06-13 20:08:55,2022-06-13 20:08:56,32.290180,-64.996666,4558.09,0.764706,0.662404,-0.033248,-0.475703,0.081841,0.055,0.183946,0.391
11986,2022-06-13 20:08:56,2022-06-13 20:08:57,32.289203,-64.997413,4565.53,0.764706,0.662404,-0.033248,-0.475703,0.081841,0.055,0.183946,0.391
11987,2022-06-13 20:08:57,2022-06-13 20:08:58,32.288204,-64.998124,4572.98,0.764706,0.662404,-0.033248,-0.475703,0.081841,0.055,0.183946,0.391
11988,2022-06-13 20:08:58,2022-06-13 20:08:59,32.287185,-64.998799,4580.36,0.181818,0.428571,0.246753,0.209647,-0.066790,0.018,0.183673,0.539


In [93]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY13b_subset.GPS_Altitude_THORNHILL.min(), DAY13b_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY13b_subset['Bins'] = pd.cut(DAY13b_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY13b_subset_1000 = DAY13b_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY13b_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY13b_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_13b = DAY13b_subset_1000[start_index:stop_index]
DAY1000_subset_13b


,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(105.59, 150.409]",2022-06-13 19:46:21.476190464,2022-06-13 19:46:22.476190464,32.235045,-64.990851,121.140833,-0.003866,0.915157,0.083644,-0.017439,0.022504,-0.040071,0.039963,0.785619
"(150.409, 195.228]",2022-06-13 19:46:50.000000000,2022-06-13 19:46:51.000000000,32.223206,-64.962163,172.722000,-0.093239,0.992661,0.063412,-0.008282,0.045448,-0.037400,0.348200,0.654400
"(195.228, 240.047]",2022-06-13 19:47:03.500000000,2022-06-13 19:47:04.500000000,32.226886,-64.946746,218.451667,0.200472,0.666274,-0.012972,0.169811,-0.023585,-0.007000,-0.041176,0.848000
"(240.047, 284.866]",2022-06-13 19:47:16.500000000,2022-06-13 19:47:17.500000000,32.235473,-64.934988,261.591786,0.200472,0.666274,-0.012972,0.169811,-0.023585,-0.007000,-0.041176,0.848000
"(284.866, 329.686]",2022-06-13 19:47:33.000000000,2022-06-13 19:47:34.000000000,32.250778,-64.927901,307.752895,0.212531,0.699017,0.020885,0.082310,-0.014742,-0.025000,-0.144509,0.814000
"(329.686, 374.505]",2022-06-13 19:47:48.000000000,2022-06-13 19:47:49.000000000,32.265570,-64.930424,352.170000,0.212531,0.699017,0.020885,0.082310,-0.014742,-0.025000,-0.144509,0.814000
"(374.505, 419.324]",2022-06-13 19:48:00.500000000,2022-06-13 19:48:01.500000000,32.275624,-64.938790,397.117857,0.008565,0.989293,-0.122056,0.122056,0.002141,-0.093000,-23.250000,0.467000
"(419.324, 464.143]",2022-06-13 19:48:11.500000000,2022-06-13 19:48:12.500000000,32.281952,-64.948895,442.846250,0.008565,0.989293,-0.122056,0.122056,0.002141,-0.093000,-23.250000,0.467000
"(464.143, 508.962]",2022-06-13 19:48:21.500000000,2022-06-13 19:48:22.500000000,32.284827,-64.959460,489.259167,-0.016953,1.034411,-0.085148,0.057763,0.009926,-0.060667,-15.555556,0.429000


In [94]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY13b_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY13b_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY13b_subset_3000 = DAY13b_subset_1000[start_index:stop_index]
DAY13b_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(1001.972, 1046.791]",2022-06-13 19:51:08.500000000,2022-06-13 19:51:09.500000000,32.277795,-64.944534,1023.430000,0.174879,0.624155,-0.041546,0.250242,-0.007729,-0.027000,-0.149171,1.035000
"(1046.791, 1091.61]",2022-06-13 19:51:18.500000000,2022-06-13 19:51:19.500000000,32.285446,-64.953669,1069.449000,0.174879,0.624155,-0.041546,0.250242,-0.007729,-0.027000,-0.149171,1.035000
"(1091.61, 1136.429]",2022-06-13 19:51:28.500000000,2022-06-13 19:51:29.500000000,32.290798,-64.964661,1113.492000,0.207700,0.650456,0.029382,0.088146,0.024316,0.027000,0.131707,0.987000
"(1136.429, 1181.248]",2022-06-13 19:51:38.500000000,2022-06-13 19:51:39.500000000,32.293438,-64.976744,1163.157000,0.207700,0.650456,0.029382,0.088146,0.024316,0.027000,0.131707,0.987000
"(1181.248, 1226.067]",2022-06-13 19:51:52.000000000,2022-06-13 19:51:53.000000000,32.292299,-64.993297,1203.252353,0.260163,0.668838,0.026178,0.024429,0.020392,0.018176,0.087233,0.986294
"(1226.067, 1270.887]",2022-06-13 19:52:08.500000000,2022-06-13 19:52:09.500000000,32.284282,-65.011196,1245.613750,0.356345,0.702538,0.020305,-0.092386,0.013198,0.002000,0.005698,0.985000
"(1270.887, 1315.706]",2022-06-13 19:52:25.000000000,2022-06-13 19:52:26.000000000,32.270083,-65.022916,1291.343529,0.049641,0.881812,0.032257,-0.018049,0.054339,-0.021824,0.177795,0.772176
"(1315.706, 1360.525]",2022-06-13 19:52:41.000000000,2022-06-13 19:52:42.000000000,32.253286,-65.026168,1340.182667,-0.222985,1.041166,0.042882,0.048027,0.090909,-0.043000,0.330769,0.583000
"(1360.525, 1405.344]",2022-06-13 19:52:54.500000000,2022-06-13 19:52:55.500000000,32.239093,-65.022448,1385.006667,-0.121134,0.871134,0.053810,0.125942,0.070248,-0.027000,0.558242,0.654500


In [95]:
DAY14 = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220614_R0_20230614T111638.ict')
start_index = np.where(DAY14["Time_Start"]==pd.to_datetime("2022-06-14 16:05:00"))
stop_index = np.where(DAY14["Time_Start"]==pd.to_datetime("2022-06-14 16:25:00"))
DAY14_subset = DAY14[start_index[0][0]:stop_index[0][0]]
DAY14_subset = DAY14_subset.replace(-888888, 0)
DAY14_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
9363,2022-06-14 16:05:00,2022-06-14 16:05:01,30.642555,-64.394685,117.000,0.419,0.764,0.052,0.223,0.006,0.024
9364,2022-06-14 16:05:01,2022-06-14 16:05:02,30.641725,-64.395343,117.085,0.419,0.764,0.052,0.223,0.006,0.024
9365,2022-06-14 16:05:02,2022-06-14 16:05:03,30.640923,-64.396042,117.190,0.419,0.764,0.052,0.223,0.006,0.024
9366,2022-06-14 16:05:03,2022-06-14 16:05:04,30.640148,-64.396780,117.295,0.238,0.747,0.077,-0.117,-0.008,0.020
9367,2022-06-14 16:05:04,2022-06-14 16:05:05,30.639403,-64.397555,117.380,0.238,0.747,0.077,-0.117,-0.008,0.020
...,...,...,...,...,...,...,...,...,...,...,...
10558,2022-06-14 16:24:55,2022-06-14 16:24:56,30.850057,-64.439182,8454.590,0.229,0.155,0.206,0.466,-0.023,-0.089
10559,2022-06-14 16:24:56,2022-06-14 16:24:57,30.851471,-64.439393,8461.000,0.229,0.155,0.206,0.466,-0.023,-0.089
10560,2022-06-14 16:24:57,2022-06-14 16:24:58,30.852885,-64.439595,8467.420,0.229,0.155,0.206,0.466,-0.023,-0.089
10561,2022-06-14 16:24:58,2022-06-14 16:24:59,30.854300,-64.439786,8473.860,0.229,0.155,0.206,0.466,-0.023,-0.089


In [96]:
normal14 = (DAY14_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY14_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY14_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY14_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY14_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY14_subset['mz44'] = DAY14_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY14_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY14_subset.loc[:, start_col2:stop_col2] = DAY14_subset.loc[:, start_col2:stop_col2].div(normal14, axis=0)
DAY14_subset['AMS'] = normal14
DAY14_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
9363,2022-06-14 16:05:00,2022-06-14 16:05:01,30.642555,-64.394685,117.000,0.286202,0.521858,0.035519,0.152322,0.004098,0.024,0.057279,1.464
9364,2022-06-14 16:05:01,2022-06-14 16:05:02,30.641725,-64.395343,117.085,0.286202,0.521858,0.035519,0.152322,0.004098,0.024,0.057279,1.464
9365,2022-06-14 16:05:02,2022-06-14 16:05:03,30.640923,-64.396042,117.190,0.286202,0.521858,0.035519,0.152322,0.004098,0.024,0.057279,1.464
9366,2022-06-14 16:05:03,2022-06-14 16:05:04,30.640148,-64.396780,117.295,0.254002,0.797225,0.082177,-0.124867,-0.008538,0.020,0.084034,0.937
9367,2022-06-14 16:05:04,2022-06-14 16:05:05,30.639403,-64.397555,117.380,0.254002,0.797225,0.082177,-0.124867,-0.008538,0.020,0.084034,0.937
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10558,2022-06-14 16:24:55,2022-06-14 16:24:56,30.850057,-64.439182,8454.590,0.221684,0.150048,0.199419,0.451113,-0.022265,-0.089,-0.388646,1.033
10559,2022-06-14 16:24:56,2022-06-14 16:24:57,30.851471,-64.439393,8461.000,0.221684,0.150048,0.199419,0.451113,-0.022265,-0.089,-0.388646,1.033
10560,2022-06-14 16:24:57,2022-06-14 16:24:58,30.852885,-64.439595,8467.420,0.221684,0.150048,0.199419,0.451113,-0.022265,-0.089,-0.388646,1.033
10561,2022-06-14 16:24:58,2022-06-14 16:24:59,30.854300,-64.439786,8473.860,0.221684,0.150048,0.199419,0.451113,-0.022265,-0.089,-0.388646,1.033


In [97]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY14_subset.GPS_Altitude_THORNHILL.min(), DAY14_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY14_subset['Bins'] = pd.cut(DAY14_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY14_subset_1000 = DAY14_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY14_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY14_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_14 = DAY14_subset_1000[start_index:stop_index]
DAY1000_subset_14


,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(117.0, 200.633]",2022-06-14 16:05:20.000,2022-06-14 16:05:21.000,30.635158,-64.414087,129.846923,0.240489,0.750211,0.058671,-0.045755,-0.003617,0.015128,0.061638,1.094641
"(200.633, 284.267]",2022-06-14 16:05:44.500,2022-06-14 16:05:45.500,30.642516,-64.438424,243.369000,0.188290,0.654691,-0.009315,0.156354,0.009980,-0.002000,-0.007067,1.503000
"(284.267, 367.9]",2022-06-14 16:05:55.000,2022-06-14 16:05:56.000,30.651917,-64.443640,326.170000,0.188290,0.654691,-0.009315,0.156354,0.009980,-0.002000,-0.007067,1.503000
"(367.9, 451.533]",2022-06-14 16:06:05.500,2022-06-14 16:06:06.500,30.662338,-64.443667,410.024500,0.136580,0.636318,0.015595,0.172636,0.038871,-0.044300,-0.258601,1.326600
"(451.533, 535.166]",2022-06-14 16:06:16.000,2022-06-14 16:06:17.000,30.671993,-64.438621,492.827273,0.130834,0.634277,0.018363,0.174445,0.042081,-0.049000,-0.286550,1.307000
"(535.166, 618.8]",2022-06-14 16:06:27.500,2022-06-14 16:06:28.500,30.680434,-64.429290,577.681667,0.078026,0.691967,0.020897,0.176373,0.032737,-0.048667,-0.179894,1.208333
"(618.8, 702.433]",2022-06-14 16:06:38.500,2022-06-14 16:06:39.500,30.684512,-64.417261,662.666500,-0.186014,0.980420,0.033566,0.186014,-0.013986,-0.047000,0.353383,0.715000
"(702.433, 786.066]",2022-06-14 16:06:48.500,2022-06-14 16:06:49.500,30.683733,-64.405511,743.799500,-0.186014,0.980420,0.033566,0.186014,-0.013986,-0.047000,0.353383,0.715000
"(786.066, 869.7]",2022-06-14 16:06:59.500,2022-06-14 16:07:00.500,30.678183,-64.394419,826.046250,0.039787,0.844581,-0.000967,0.117121,-0.000522,-0.027333,0.244935,0.767000


In [98]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY14_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY14_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY14_subset_3000 = DAY14_subset_1000[start_index:stop_index]
DAY14_subset_3000.describe()

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
count,28,28,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000
mean,2022-06-14 16:09:40.571428352,2022-06-14 16:09:41.571428864,30.665512,-64.406044,2207.918130,0.663666,-0.504677,-0.190825,1.074114,-0.042278,0.006289,-0.075609,0.512529
min,2022-06-14 16:07:29.500000,2022-06-14 16:07:30.500000,30.637684,-64.436561,1077.070000,-0.666667,-5.166667,-2.600000,-0.237721,-0.609756,-0.054000,-0.950000,-0.041000
25%,2022-06-14 16:08:34.124999936,2022-06-14 16:08:35.124999936,30.644458,-64.425179,1642.586361,0.355556,-0.188298,-0.051744,-0.155809,-0.004442,-0.019000,-0.120521,0.351000
50%,2022-06-14 16:09:38.750000128,2022-06-14 16:09:39.750000128,30.661879,-64.407742,2206.432056,0.569829,0.300761,0.008401,0.116751,0.015429,0.013000,0.037249,0.509000
75%,2022-06-14 16:10:45.624999936,2022-06-14 16:10:46.624999936,30.684682,-64.388273,2773.432083,0.685658,0.431226,0.050980,0.341280,0.040151,0.039000,0.084967,0.788000
max,2022-06-14 16:11:56,2022-06-14 16:11:57,30.699582,-64.368152,3338.431111,2.682927,0.583497,0.756098,9.433333,0.100000,0.050000,0.238579,1.037000
std,NaN,NaN,0.021283,0.022013,688.131611,0.741957,1.732795,0.781668,2.698247,0.186442,0.033700,0.306940,0.347994


In [99]:
DAY17 = simple_read('C:/Users/taiwoajayi/Desktop/chem/activate-mrg1_hu25_20220617_R0_20230614T111638.ict')
start_index = np.where(DAY17["Time_Start"]==pd.to_datetime("2022-06-17 16:20:00"))
stop_index = np.where(DAY17["Time_Start"]==pd.to_datetime("2022-06-17 16:47:00")) 
DAY17_subset = DAY17[start_index[0][0]:stop_index[0][0]]
DAY17_subset = DAY17_subset.replace(-888888, 0)
DAY17_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA
10469,2022-06-17 16:20:00,2022-06-17 16:20:01,32.206574,-65.006772,134.100,0.504,0.614,0.000,0.096,0.051,0.016
10470,2022-06-17 16:20:01,2022-06-17 16:20:02,32.206131,-65.005761,133.795,0.504,0.614,0.000,0.096,0.051,0.016
10471,2022-06-17 16:20:02,2022-06-17 16:20:03,32.205725,-65.004728,133.485,0.504,0.614,0.000,0.096,0.051,0.016
10472,2022-06-17 16:20:03,2022-06-17 16:20:04,32.205357,-65.003676,133.170,0.504,0.614,0.000,0.096,0.051,0.016
10473,2022-06-17 16:20:04,2022-06-17 16:20:05,32.205027,-65.002606,132.825,0.504,0.614,0.000,0.096,0.051,0.016
...,...,...,...,...,...,...,...,...,...,...,...
12084,2022-06-17 16:46:55,2022-06-17 16:46:56,32.158601,-65.020304,7639.050,0.540,0.148,-0.076,0.036,0.043,0.004
12085,2022-06-17 16:46:56,2022-06-17 16:46:57,32.157293,-65.019563,7645.810,0.540,0.148,-0.076,0.036,0.043,0.004
12086,2022-06-17 16:46:57,2022-06-17 16:46:58,32.156002,-65.018778,7652.610,0.540,0.148,-0.076,0.036,0.043,0.004
12087,2022-06-17 16:46:58,2022-06-17 16:46:59,32.154729,-65.017953,7659.440,0.540,0.148,-0.076,0.036,0.043,0.004


In [100]:
normal17 = (DAY17_subset['Chl_Ave_IsoK_STP_ZIEMBA'] + DAY17_subset['Org_Ave_IsoK_STP_ZIEMBA'] +
          DAY17_subset['SO4_Ave_IsoK_STP_ZIEMBA'] + DAY17_subset['NO3_Ave_IsoK_STP_ZIEMBA'] +
          DAY17_subset['NH4_Ave_IsoK_STP_ZIEMBA'])
# Create a new column 'mz44' by dividing 'mz44_Ave_IsoK_STP_ZIEMBA' by 'Org_Ave_IsoK_STP_ZIEMBA'
DAY17_subset['mz44'] = DAY17_subset['mz44_Ave_IsoK_STP_ZIEMBA'] / DAY17_subset['Org_Ave_IsoK_STP_ZIEMBA']

# Specify the range of columns for division
start_col2 = 'Org_Ave_IsoK_STP_ZIEMBA'
stop_col2 = 'Chl_Ave_IsoK_STP_ZIEMBA'

# Perform division and assign the result to the specified range of columns
DAY17_subset.loc[:, start_col2:stop_col2] = DAY17_subset.loc[:, start_col2:stop_col2].div(normal17, axis=0)
DAY17_subset['AMS'] = normal17
DAY17_subset

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
10469,2022-06-17 16:20:00,2022-06-17 16:20:01,32.206574,-65.006772,134.100,0.398419,0.485375,0.000000,0.075889,0.040316,0.016,0.031746,1.265
10470,2022-06-17 16:20:01,2022-06-17 16:20:02,32.206131,-65.005761,133.795,0.398419,0.485375,0.000000,0.075889,0.040316,0.016,0.031746,1.265
10471,2022-06-17 16:20:02,2022-06-17 16:20:03,32.205725,-65.004728,133.485,0.398419,0.485375,0.000000,0.075889,0.040316,0.016,0.031746,1.265
10472,2022-06-17 16:20:03,2022-06-17 16:20:04,32.205357,-65.003676,133.170,0.398419,0.485375,0.000000,0.075889,0.040316,0.016,0.031746,1.265
10473,2022-06-17 16:20:04,2022-06-17 16:20:05,32.205027,-65.002606,132.825,0.398419,0.485375,0.000000,0.075889,0.040316,0.016,0.031746,1.265
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12084,2022-06-17 16:46:55,2022-06-17 16:46:56,32.158601,-65.020304,7639.050,0.781476,0.214182,-0.109986,0.052098,0.062229,0.004,0.007407,0.691
12085,2022-06-17 16:46:56,2022-06-17 16:46:57,32.157293,-65.019563,7645.810,0.781476,0.214182,-0.109986,0.052098,0.062229,0.004,0.007407,0.691
12086,2022-06-17 16:46:57,2022-06-17 16:46:58,32.156002,-65.018778,7652.610,0.781476,0.214182,-0.109986,0.052098,0.062229,0.004,0.007407,0.691
12087,2022-06-17 16:46:58,2022-06-17 16:46:59,32.154729,-65.017953,7659.440,0.781476,0.214182,-0.109986,0.052098,0.062229,0.004,0.007407,0.691


In [101]:
#binning data into groups
nbins = 100
bins_1000 = np.linspace(DAY17_subset.GPS_Altitude_THORNHILL.min(), DAY17_subset.GPS_Altitude_THORNHILL.max(), nbins+1)
DAY17_subset['Bins'] = pd.cut(DAY17_subset['GPS_Altitude_THORNHILL'], bins_1000)
DAY17_subset_1000 = DAY17_subset.groupby('Bins').mean()
#locating the altitude of the spiral
start_altitude = 0
stop_altitude = 1000

start_index = np.where(DAY17_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY17_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY1000_subset_17 = DAY17_subset_1000[start_index:stop_index]
DAY1000_subset_17


,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(124.73, 200.146]",2022-06-17 16:20:35.295774720,2022-06-17 16:20:36.295774720,32.218829,-64.978554,138.339507,0.350552,0.413774,0.019090,0.195715,0.020870,0.022859,0.051479,1.456042
"(200.146, 275.561]",2022-06-17 16:21:24.500000256,2022-06-17 16:21:25.500000000,32.250650,-64.989881,236.188462,0.466724,0.509075,0.055315,-0.033708,0.002593,0.023000,0.042593,1.157000
"(275.561, 350.977]",2022-06-17 16:21:50.000000000,2022-06-17 16:21:51.000000000,32.242129,-65.012285,309.913800,0.476593,0.415212,-0.000941,0.116602,-0.007465,0.095000,0.131290,1.507400
"(350.977, 426.392]",2022-06-17 16:22:13.000000000,2022-06-17 16:22:14.000000000,32.223573,-65.013314,393.872857,0.449196,0.493224,0.038380,0.046572,-0.027373,0.075857,0.122088,1.369143
"(426.392, 501.808]",2022-06-17 16:22:35.000000000,2022-06-17 16:22:36.000000000,32.213844,-64.995274,463.940435,0.360784,0.576891,0.050621,0.046336,-0.034632,0.035696,0.050153,1.189826
"(501.808, 577.223]",2022-06-17 16:22:57.000000000,2022-06-17 16:22:58.000000000,32.221232,-64.974504,541.829286,0.240557,0.656064,0.043738,0.093439,-0.033797,-0.013000,-0.053719,1.006000
"(577.223, 652.638]",2022-06-17 16:23:17.000000000,2022-06-17 16:23:18.000000000,32.239178,-64.968086,615.008684,0.290290,0.446446,-0.023524,0.297297,-0.010511,0.043000,0.074138,1.998000
"(652.638, 728.054]",2022-06-17 16:23:38.000000000,2022-06-17 16:23:39.000000000,32.257136,-64.980597,686.849130,0.402768,0.386191,-0.009062,0.223379,-0.003276,0.069087,0.075691,2.199391
"(728.054, 803.47]",2022-06-17 16:23:59.000000000,2022-06-17 16:24:00.000000000,32.263103,-65.005527,763.259474,0.504717,0.332443,0.002665,0.155868,0.004307,0.091000,0.075620,2.381316


In [102]:
#locating the altitude of the spiral
start_altitude = 1001
stop_altitude = 3500

start_index = np.where(DAY17_subset_1000['GPS_Altitude_THORNHILL'] >= start_altitude)[0][0]
stop_index = np.where(DAY17_subset_1000['GPS_Altitude_THORNHILL'] <= stop_altitude)[0][-1]

DAY17_subset_3000 = DAY17_subset_1000[start_index:stop_index]
DAY17_subset_3000

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
Bins,,,,,,,,,,,,,
"(1029.716, 1105.132]",2022-06-17 16:25:25.000000000,2022-06-17 16:25:26.000000000,32.198372,-65.013058,1069.572000,0.356109,0.460788,0.017503,0.158532,0.007068,0.091000,0.086011,2.971000
"(1105.132, 1180.547]",2022-06-17 16:25:50.500000000,2022-06-17 16:25:51.500000000,32.205940,-64.983412,1143.704231,0.360798,0.622008,0.003090,0.003680,0.010424,0.032346,0.043484,2.026769
"(1180.547, 1255.962]",2022-06-17 16:27:55.000000000,2022-06-17 16:27:56.000000256,32.227871,-65.006612,1209.486996,0.338093,0.585762,-0.005596,0.082732,-0.000991,0.062328,0.100733,1.806142
"(1255.962, 1331.378]",2022-06-17 16:29:59.500000000,2022-06-17 16:30:00.500000000,32.252481,-65.015879,1291.048462,0.216365,0.519578,0.015060,0.219880,0.029116,-0.004000,-0.009281,1.992000
"(1331.378, 1406.794]",2022-06-17 16:30:25.000000000,2022-06-17 16:30:26.000000000,32.239553,-65.045051,1370.451200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1406.794, 1482.209]",2022-06-17 16:30:52.000000000,2022-06-17 16:30:53.000000000,32.210442,-65.051537,1445.753448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"(1482.209, 1557.625]",2022-06-17 16:31:19.999999744,2022-06-17 16:31:21.000000000,32.188000,-65.027798,1519.431852,0.453729,0.497238,-0.017956,0.069751,-0.002762,0.003000,0.004566,1.448000
"(1557.625, 1633.04]",2022-06-17 16:31:47.000000000,2022-06-17 16:31:48.000000000,32.191553,-64.994280,1593.622963,0.260323,0.712381,0.010336,0.029506,-0.012547,0.004630,0.008362,2.309259
"(1633.04, 1708.456]",2022-06-17 16:32:14.000000000,2022-06-17 16:32:15.000000256,32.215972,-64.975734,1668.389259,0.203449,0.691364,0.031235,0.082081,-0.008129,-0.033000,-0.059172,2.721741


In [103]:

# Concatenate the dataframes vertically (along rows)
ocean = pd.concat([DAY1000_subset_1000, DAY1000_subset_1000b], axis=0)

# Reset the index of the merged dataframe
ocean.reset_index(drop=True, inplace=True)
ocean.describe()

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
count,54,54,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,2022-06-02 15:57:34.006548480,2022-06-02 15:57:35.006548480,32.260783,-65.086951,539.226840,0.132150,0.660084,-0.000605,0.206228,0.002144,0.004793,-0.000221,2.009781
min,2022-06-02 13:54:46.487500032,2022-06-02 13:54:47.487500032,32.208589,-65.305158,121.847785,-0.150927,0.562405,-0.058308,0.024961,-0.022809,-0.073000,-1.258621,1.186000
25%,2022-06-02 13:58:04.475000064,2022-06-02 13:58:05.475000064,32.245279,-65.170465,333.169000,0.080422,0.593037,-0.005327,0.166816,-0.008058,-0.023959,0.007440,1.757400
50%,2022-06-02 13:58:43.794871808,2022-06-02 13:58:44.794871808,32.259518,-65.050124,542.367759,0.154774,0.650224,0.005841,0.223289,0.001638,0.010857,0.053174,2.024623
75%,2022-06-02 18:38:26.124999936,2022-06-02 18:38:27.124999936,32.279729,-65.013358,748.823929,0.197439,0.709264,0.015436,0.236161,0.013219,0.035124,0.100372,2.288898
max,2022-06-02 18:41:01.500000,2022-06-02 18:41:02.500000,32.302282,-64.962424,951.478333,0.312242,0.814503,0.036235,0.409781,0.026932,0.068000,0.589286,2.584480
std,NaN,NaN,0.022520,0.096509,245.995329,0.098046,0.077543,0.025203,0.076740,0.013519,0.037916,0.317933,0.352029


In [104]:
# Concatenate the dataframes vertically (along rows)
ocean2 = pd.concat([DAY3000_subset_3000, DAY3000_subset_3000b], axis=0)

# Reset the index of the merged dataframe
ocean2.reset_index(drop=True, inplace=True)
ocean2.describe()

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
count,136,136,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000
mean,2022-06-02 16:27:31.669278976,2022-06-02 16:27:32.669278976,32.280743,-65.038310,2074.413541,0.419868,0.427455,-0.011056,0.153400,0.010332,0.026926,-0.020333,1.176570
min,2022-06-02 13:50:07.969465600,2022-06-02 13:50:08.969465600,32.218661,-65.750983,1005.609375,-0.045339,0.075472,-0.246862,-1.257669,-0.147239,-0.064400,-0.612438,0.163000
25%,2022-06-02 14:05:01.409091072,2022-06-02 14:05:02.409091072,32.259912,-65.052320,1527.248239,0.265647,0.316354,-0.042852,0.130687,-0.005044,-0.018417,-0.103912,0.854273
50%,2022-06-02 18:41:27.249999872,2022-06-02 18:41:28.249999872,32.277253,-65.004105,2052.482500,0.402050,0.373640,-0.000256,0.229759,0.012840,0.019182,0.036010,1.296000
75%,2022-06-02 18:47:16,2022-06-02 18:47:17,32.300794,-64.961800,2582.140000,0.537884,0.573743,0.030583,0.280397,0.026818,0.071777,0.112311,1.552031
max,2022-06-02 18:52:12,2022-06-02 18:52:13,32.365645,-64.908365,3434.677778,1.533742,0.950920,0.070674,0.490960,0.093484,0.182000,0.234899,1.892000
std,NaN,NaN,0.031836,0.125763,651.241184,0.269174,0.184249,0.061932,0.257753,0.034785,0.054627,0.190098,0.470202


In [105]:
# Concatenate the dataframes vertically (along rows)
North_America = pd.concat([DAY1000_subset_7, DAY1000_subset_7b, DAY1000_subset_17], axis=0)

# Reset the index of the merged dataframe
North_America.reset_index(drop=True, inplace=True)
North_America.describe()

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
count,67,67,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000
mean,2022-06-09 07:55:27.363589120,2022-06-09 07:55:28.363588864,32.231992,-65.009280,548.337133,0.530363,0.322610,0.026716,0.117130,0.003182,0.187758,0.109430,2.820057
min,2022-06-07 14:32:05.300000,2022-06-07 14:32:06.300000,32.170582,-65.065238,134.321500,0.240557,0.231022,-0.024861,-0.033708,-0.034632,-0.013000,-0.053719,1.006000
25%,2022-06-07 14:35:12,2022-06-07 14:35:13,32.204178,-65.035459,340.337833,0.528037,0.263685,0.016093,0.087474,-0.002753,0.077000,0.088829,1.774909
50%,2022-06-07 19:08:21.500000,2022-06-07 19:08:22.500000,32.234441,-65.004622,544.358929,0.547859,0.295658,0.023642,0.129223,0.004268,0.141000,0.117227,2.381316
75%,2022-06-07 19:11:12,2022-06-07 19:11:13,32.255360,-64.980962,761.335779,0.564268,0.351176,0.034905,0.156838,0.007958,0.296000,0.131660,3.964545
max,2022-06-17 16:24:41.500000,2022-06-17 16:24:42.500000,32.302506,-64.968086,975.912308,0.632981,0.656064,0.078296,0.297297,0.029431,0.372000,0.162281,4.444000
std,NaN,NaN,0.036142,0.028948,245.678658,0.072524,0.082097,0.019547,0.061616,0.012060,0.118788,0.038142,1.151343


In [106]:
# Concatenate the dataframes vertically (along rows)
North_America2 = pd.concat([DAY7_subset_3000, DAY7b_subset_3000, DAY17_subset_3000], axis=0)

# Reset the index of the merged dataframe
North_America2.reset_index(drop=True, inplace=True)
North_America2.describe()

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
count,171,171,171.000000,171.000000,171.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000,159.000000
mean,2022-06-09 13:56:08.766082304,2022-06-09 13:56:09.766081792,32.233562,-65.015713,2106.253110,0.577069,0.253687,0.010314,0.155217,0.003713,0.085741,0.102339,1.498461
min,2022-06-07 14:38:12,2022-06-07 14:38:13,32.162806,-65.062171,1003.411000,0.198009,0.085057,-0.104762,-0.518234,-0.073041,-0.049000,-0.087971,0.465379
25%,2022-06-07 14:44:39.249999872,2022-06-07 14:44:40.249999872,32.210141,-65.045180,1545.078333,0.465710,0.163000,-0.015538,0.069863,-0.012803,0.041000,0.056204,0.919000
50%,2022-06-07 19:15:57.500000,2022-06-07 19:15:58.500000,32.236307,-65.016066,2079.076250,0.572480,0.191280,0.017021,0.171790,0.005335,0.079000,0.115595,1.372000
75%,2022-06-07 19:21:51.500000,2022-06-07 19:21:52.500000,32.253539,-64.988696,2612.969500,0.682184,0.289445,0.031233,0.255196,0.015078,0.123000,0.132333,1.881000
max,2022-06-17 16:38:08.500000,2022-06-17 16:38:09.500000,32.300554,-64.960247,3459.904000,0.896848,0.712381,0.158824,0.537964,0.079365,0.239000,0.352724,3.410000
std,NaN,NaN,0.034153,0.030088,664.456867,0.160408,0.133880,0.051033,0.162290,0.028967,0.056974,0.059171,0.772654


In [107]:
# Concatenate the dataframes vertically (along rows)
Caribbean = pd.concat([DAY1000_subset_3, DAY1000_subset_8, DAY1000_subset_10, DAY1000_subset_10b, DAY1000_subset_11, DAY1000_subset_11b], axis=0)

# Reset the index of the merged dataframe
Caribbean.reset_index(drop=True, inplace=True)
Caribbean.describe()

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
count,123,123,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000
mean,2022-06-09 21:31:55.508952320,2022-06-09 21:31:56.508952064,32.144896,-65.350799,537.555141,0.227407,0.608713,0.008742,0.153348,0.001790,0.037395,0.027952,1.857347
min,2022-06-03 13:45:28.789007104,2022-06-03 13:45:29.789007104,31.035566,-67.436618,121.109551,-0.007000,0.462028,-0.041004,-0.087434,-0.026037,-0.065333,-1.516820,1.034200
25%,2022-06-08 20:33:55.500000,2022-06-08 20:33:56.500000,31.372011,-65.838736,332.799583,0.159278,0.559833,-0.004643,0.114253,-0.004505,0.000000,-0.007376,1.565000
50%,2022-06-10 19:58:13.842105088,2022-06-10 19:58:14.842105088,32.243637,-65.035595,538.712500,0.216404,0.622156,0.013814,0.173765,0.000946,0.028000,0.067308,1.760000
75%,2022-06-11 15:28:22.249999872,2022-06-11 15:28:23.249999872,32.297915,-64.986404,745.678750,0.299712,0.651947,0.018752,0.205543,0.007599,0.068500,0.128463,2.135909
max,2022-06-11 20:10:12.500000,2022-06-11 20:10:13.500000,34.214933,-64.424176,961.682500,0.430960,0.869809,0.098426,0.291488,0.043084,0.168000,0.808511,2.933000
std,NaN,NaN,0.834868,0.817590,242.594382,0.105720,0.073749,0.020064,0.086661,0.009024,0.049361,0.259279,0.435343


In [108]:
# Concatenate the dataframes vertically (along rows)
Caribbean2 = pd.concat([DAY3_subset_3000, DAY8_subset_3000, DAY10_subset_3000, DAY10b_subset_3000, DAY11_subset_3000, DAY11b_subset_3000, DAY13_subset_3000, DAY1000_subset_13b], axis=0)

# Reset the index of the merged dataframe
Caribbean2.reset_index(drop=True, inplace=True)
Caribbean2.describe()

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
count,431,431,431.000000,431.000000,431.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000,430.000000
mean,2022-06-10 11:56:14.973697280,2022-06-10 11:56:15.973697792,32.145916,-65.235505,2137.176787,0.336651,0.517821,0.008033,0.133673,0.003821,0.035998,0.658321,1.049821
min,2022-06-03 13:53:27.769230848,2022-06-03 13:53:28.769230592,31.036776,-67.516110,121.140833,-0.335095,0.163621,-0.147849,-0.894161,-0.288241,-0.133000,-23.250000,0.274000
25%,2022-06-08 20:43:57.750000128,2022-06-08 20:43:58.750000128,31.594612,-65.095014,1520.495568,0.206002,0.402655,-0.018737,0.059212,-0.008606,-0.006000,-0.037110,0.590750
50%,2022-06-10 20:09:28.380952320,2022-06-10 20:09:29.380952320,32.237935,-65.027415,2152.079803,0.332969,0.508663,0.008661,0.146986,0.005535,0.022000,0.100457,0.923000
75%,2022-06-11 20:17:06.958333440,2022-06-11 20:17:07.958333440,32.279986,-64.979378,2784.418750,0.458824,0.590430,0.032452,0.227730,0.021686,0.083000,0.165139,1.357094
max,2022-06-13 19:50:47,2022-06-13 19:50:48,34.194709,-64.393900,3448.995000,1.721138,1.201511,0.213045,0.630641,0.137045,0.212000,133.000000,3.293000
std,NaN,NaN,0.684266,0.641941,769.502910,0.232903,0.177128,0.052493,0.188596,0.041449,0.061891,9.678631,0.602854


In [109]:
Africa = pd.concat([DAY1000_subset_13, DAY1000_subset_13b, DAY1000_subset_14])
# Reset the index of the merged dataframe
Africa.reset_index(drop=True, inplace=True)
Africa.describe()

,Time_Start,Time_Stop,Latitude_THORNHILL,Longitude_THORNHILL,GPS_Altitude_THORNHILL,Org_Ave_IsoK_STP_ZIEMBA,SO4_Ave_IsoK_STP_ZIEMBA,NO3_Ave_IsoK_STP_ZIEMBA,NH4_Ave_IsoK_STP_ZIEMBA,Chl_Ave_IsoK_STP_ZIEMBA,mz44_Ave_IsoK_STP_ZIEMBA,mz44,AMS
count,48,48,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,2022-06-13 21:57:17.503670528,2022-06-13 21:57:18.503670784,31.918298,-64.885146,541.348157,0.174209,0.730000,-0.006602,0.093138,0.009255,-0.014565,-1.338109,0.991408
min,2022-06-13 14:30:32.200000,2022-06-13 14:30:33.200000,30.635158,-65.097735,121.140833,-0.186014,0.524390,-0.147849,-0.381720,-0.041593,-0.093000,-23.250000,0.353000
25%,2022-06-13 14:33:19.249999872,2022-06-13 14:33:20.249999872,32.200276,-65.034858,335.132500,0.055046,0.625708,-0.035262,0.000750,-0.013986,-0.041129,-0.196731,0.741250
50%,2022-06-13 19:47:40.500000,2022-06-13 19:47:41.500000,32.235259,-64.988673,545.363575,0.188290,0.662885,-0.001057,0.132435,0.002141,-0.016042,-0.041176,0.872000
75%,2022-06-13 19:50:19,2022-06-13 19:50:20,32.266888,-64.933701,747.631396,0.265443,0.860058,0.033566,0.179802,0.032919,0.012000,0.086220,1.313000
max,2022-06-14 16:07:10.500000,2022-06-14 16:07:11.500000,32.287260,-64.387230,954.156429,0.491389,1.124646,0.083644,0.366972,0.077957,0.061000,0.353383,1.804000
std,NaN,NaN,0.649748,0.245438,247.564084,0.157141,0.167254,0.054783,0.154638,0.027245,0.038495,5.136940,0.384486


In [111]:
# Concatenate the dataframes vertically (along rows)
merged_df = pd.concat([Caribbean.describe(), Caribbean2.describe(), North_America.describe(), North_America2.describe(), ocean.describe(), ocean2.describe(), Africa.describe(), DAY14_subset_3000.describe()], axis=0)

# Reset the index of the merged dataframe
merged_df.reset_index(drop=False, inplace=True)

# Display the merged dataframe
merged_df
merged_df.to_csv('C:/Users/taiwoajayi/OneDrive - University of Arizona/python/chemistry_mass_description.csv')